In [1]:
!pip install flask-ngrok
!pip install pyngrok

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import google.generativeai as genai
import random
import string
import os
import re
from IPython.display import Markdown

# ----------------- Configuration -----------------
app = Flask(__name__)
NGROK_AUTH_TOKEN = "YOUR_NGROK_API_KEY"
GENAI_API_KEY = "YOUR_GEMINI_API_KEY"

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(5000).public_url
genai.configure(api_key=GENAI_API_KEY)
gemini_model = genai.GenerativeModel("gemini-2.0-flash")

# ----------------- FIELDS -----------------
MANDATORY_FIELDS = ["study_type", "specialization", "field", "system_type"]
OPTIONAL_FIELDS = [
    "national_direction", "project_algorithms", "target_audience", "integration",
    "programming_languages", "technologies", "database_type", "ai_data_type",
    "ai_use_types", "ai_algorithms", "initial_idea", "allow_suggested_users"
]

# ----------------- Creative Element -----------------
def get_creative_spark():
    emojis = ["💡", "✨", "🚀", "🌟", "🧠", "🎯", "📈", "🔬", "🛠️"]
    adjectives = ["مبتكر", "غير تقليدي", "رائد", "فريد", "ملهم", "استثنائي", "ذكي"]
    nouns = ["مفهوم", "حل", "نهج", "تصميم", "فكرة", "مشروع", "نظام"]
    verbs = ["ابتكر", "صمّم", "اكتشف", "طوّر", "أطلق", "استلهم", "ابدأ"]
    return f"{random.choice(emojis)} {random.choice(verbs)} {random.choice(adjectives)} {random.choice(nouns)} – رمز المشروع: {''.join(random.choices(string.ascii_uppercase + string.digits, k=5))}"

# ----------------- Prompt Builder -----------------
def build_prompt(data, missing_fields):
    creative_spark = get_creative_spark()
    filled = "\n".join([f"- **{field.replace('_', ' ').title()}**: {data[field]}" for field in data])
    missing = "\n".join([f"- **{field.replace('_', ' ').title()}**: *(🔄 برجاء توليد قيمة منطقية ومناسبة بناءً على السياق)*" for field in missing_fields])

    if 'integration' not in data or not data['integration']:
      if 'allow_suggested_users' not in data or not data['allow_suggested_users']:
          # Both 'integration' and 'allow_suggested_users' are missing or empty
          prompt = f"""
         الرجاء توليد نظام متكامل لمشروع تخرج مبتكر، يشمل جميع المكونات الأساسية لأي نظام احترافي مثل:
        • استقبال ومعالجة البيانات
        • تحليل أو تصنيف أو تنفيذ وظائف حسب المجال
        • إدارة العمليات وتتبع حالتها
        • إنشاء تقارير، إشعارات، وصلاحيات متعددة
        • تكامل مع أنظمة خارجية فقط إذا تم تحديد ذلك من قبل المستخدم
        ❗️ إذا كانت بعض الحقول غير محددة من قبل المستخدم، يُسمح للنظام باقتراح قيم مناسبة فقط لهذه الحقول المفقودة، بشرط أن تكون منطقية ومتناسبة مع نوع المشروع والمجال.
        ❌ لا يُسمح بتعديل أو استبدال أي قيمة تم تحديدها من قِبل المستخدم صراحة.
        ❌ لا يُسمح بتضمين أي تقنيات أو خوارزميات أو مفاهيم تتعلق بالذكاء الاصطناعي ما لم يتم طلبها صراحة ضمن بيانات المستخدم.
        ________________________________________
        ✅ تنسيق الرد المطلوب:
        1. **العنوان المقترح:**
        [عنوان واضح ومبتكر للفكرة]
        
        2. **وصف تفصيلي للفكرة مع أمثلة:**
        شرح شامل لطبيعة النظام، المشكلة التي يعالجها، طريقة استخدامه، الجهات المستفيدة، وسيناريوهات تطبيقية حقيقية.
        
        3. **المستخدمون ووظائفهم في النظام:**
        • يجب أن يشمل النظام "مدير النظام".
        • يدعم: تسجيل الدخول، تسجيل الخروج، إنشاء حساب جديد إن لزم.
        • لكل مستخدم: اذكر 5 وظائف واضحة على الأقل.
        o تحديد صفته، و5 وظائف له.
        • يُمنع ذكر اسماء نهائياً
        • يُمنع تكرار المستخدمين أو وظائفهم في قسم منفصل.
        
        4. **مميزات المشروع (لا تقل عن 5):**
        • مميزات عملية توضح قوة المشروع وجدواه.
        • يجب أن يحتوي النظام على ميزة مبتكرة فارقة تميّزه عن الأنظمة الأخرى المشابهة (سواء كان موقعًا أو تطبيقًا).
        • يُفضل أن تكون هذه الميزة مرتبطة بـ "علامة جودة" أو وظيفة حصرية تضيف قيمة واضحة.
        • يجب شرحها من حيث:
        o الهدف منها
        o كيف تعمل داخل النظام
        o أثرها على المستخدم أو سمعة النظام
        o سبب تميزها عن بقية الأنظمة المشابهة
        
        5. **الهدف الوطني:**
        • شرح مختصر لا يتجاوز سطرين فقط.
        • يُولد بناءً على فكرة المشروع الفعلية والمجال المستخدم.
        • يوضّح كيف يدعم المشروع فعليًا التوجه الوطني الذي حدده المستخدم (مثل: دعم التعليم الرقمي، تعزيز نمط الحياة الصحي...).
        • يجب ربطه بـ رؤية المملكة العربية السعودية 2030.
        • يُمنع تثبيت أو نسخ هدف عام لا يعكس الفكرة الفعلية.
        
        6. **شرح الخوارزميات:**
        • شرح علمي مبسط لكل خوارزمية مستخدمة (مثل: Merge Sort، K-Means).
        • توضيح كيف تُستخدم فعليًا داخل النظام.
        • إرفاق مثال تطبيقي لبيان فائدتها.
        
        7. **لغات البرمجة، التقنيات، وخوارزميات الذكاء الاصطناعي:**
        • لكل لغة أو تقنية: شرح لا يتجاوز سطرًا واحدًا يوضح وظيفتها داخل المشروع.
        • في حال تم استخدام الذكاء الاصطناعي:
        o تحديد نوع الذكاء (تحليل، تصنيف، تنبؤ...)
        o ما الذي يتم تحليله أو تصنيفه داخل النظام
        o شرح مبسط لطريقة عمل الخوارزمية
        o مثال تطبيقي يوضح فائدة الخوارزمية
        
        ________________________________________
        📌 القيود الصارمة (يجب الالتزام بها):
        • ✅ توليد نظام متكامل وظيفيًا، لا مجرد فكرة أو أداة جزئية.
        • ✅ الالتزام التام بجميع الأدوات، اللغات، والتقنيات المذكورة فقط.
        • ✅ لا يُسمح بتعديل أو تجاوز أي قيمة صرّح بها المستخدم.
        • ✅ يُسمح باقتراح القيم فقط في حال غيابها.
        • ✅ يُمنع تضمين الذكاء الاصطناعي نهائيًا ما لم يُطلب صراحة.
        • ✅ لكل مستخدم: 5 وظائف على الأقل.
        • ✅ لا يُسمح بتكرار عرض المستخدمين أو وظائفهم.
        • ✅ شرح اللغات لا يتجاوز سطرًا واحدًا.
        • ✅ الهدف الوطني لا يتجاوز سطرين، ويُولّد حسب الفكرة ويرتبط برؤية المملكة 2030.
        • ✅ يجب أن يحتوي المشروع على ميزة فارقة تميز النظام بوضوح عن الأنظمة الأخرى.
        • ✅ عدم وضع اي جمل افتتاحية او جمل ختامية في الرد النهائي
        • ✅ الالتزام بترقيم كل الفقرات والعناوين الرئيسية والعناوين الفرعية والاقسام الداخلية يتم ترقيمها بطريقة منسقة
        • ✅ يرجى تنسيق النص واضافة الايموجز والعناوين الرئيسية والفرعية بشكل منسق ع طريقة Markdown
        • ✅ يجب التاكد ان العناويين الرئيسية بحجم وخط منسقين يظهرون انهم عناويين رئيسية كذلك العناوين الفرعية والترقيم
        * ✅ برجاء عدم وضع جمل افتتاحية او ختامية يجب الالتزام بالتركيز ع توليد الفكرة مباشرة
        * ✅ يجب توليد قواعد البيانات المقترحة و التقنيات المستخدمة اذا كانت هذه القيم فارغة او لم يتم عرضها من المستخدم "technologies": "","database_type": ""
        * ✅ يجب استخدام الفكرة الاولية او initial_idea اللتي ارسلها المستخدم في توليد فكرة المشروع )مهمة جدا(
        _______________________________________
        هذه البيانات اللتي يتم بناءا عليها بناء الفكرة بكل تفاصيلها. الرجاء الالتزام بهذه المعطيات فقط
        ## 📂 البيانات المتوفرة:
        تفاصيل الفكرة :

        نوع الدراسة: {data.get("study_type", "")}
        التخصص: {data.get("specialization", "")}
        المجال: {data.get("field", "")}
        التوجه الوطني: {data.get("national_direction", "")}
        خوارزميات المشروع: {data.get("project_algorithms", "")}
        الفئة المستهدفة: {data.get("target_audience", "")}
        نوع النظام: {data.get("system_type", "")}
        لغات البرمجة: {data.get("programming_languages", "")}
        التقنيات المستخدمة: {data.get("technologies", "")}
        نوع قواعد البيانات: {data.get("database_type", "")}
        نوع البيانات المستخدمة في الذكاء الاصطناعي: {data.get("ai_data_type", "")}
        أنواع الاستخدام للذكاء الاصطناعي: {data.get("ai_use_types", "")}
        الخوارزميات المقترحة للذكاء الاصطناعي: {data.get("ai_algorithms", "")}
        ملاحظات المستخدم أو وصفه الأولي: {data.get("initial_idea", "")}
       
        هذا مثال حي يمكنك الاستفادة منه
        1. **العنوان المقترح:**
        نظام "بيانات بلس" لتحسين جودة قواعد البيانات 🌟
        
        2. **وصف تفصيلي للفكرة مع أمثلة:**
        نظام "بيانات بلس" 📊 هو تطبيق جوال مبتكر صُمم خصيصًا لمعالجة المشكلات الشائعة في جودة قواعد البيانات، مثل التكرار، البيانات غير المتسقة، والأخطاء الإملائية. هذه المشكلات تؤثر سلبًا على موثوقية البيانات وصحة القرارات المتخذة بناءً عليها. يعمل النظام عن طريق استقبال البيانات، معالجتها، ثم تحليلها لاكتشاف هذه المشاكل، وتقديم حلول مقترحة لتحسينها.
        يستطيع محللو البيانات وصناع القرار استخدام النظام لتحليل مجموعات بيانات مختلفة، مثل بيانات العملاء 🧑‍🤝‍🧑 أو المنتجات 🛍️، وتصحيح الأخطاء بشكل فعال لضمان دقة المعلومات.
        مثال 1: يمكن لشركة تجزئة 🏪 استخدام "بيانات بلس" لتنظيف قاعدة بيانات عملائها من الإدخالات المكررة أو غير الدقيقة، مما يتيح لهم استهداف العملاء بشكل أفضل وتحسين حملاتهم التسويقية 📈.
        مثال 2: سيناريو آخر هو استخدامه من قبل مؤسسة مالية 💰 للتحقق من سلامة بيانات المعاملات، وتقليل الأخطاء التي قد تؤدي إلى خسائر مالية أو عدم امتثال للوائح.
        
        3. **المستخدمون ووظائفهم في النظام:**
        **مدير النظام ⚙️:**
        إدارة حسابات المستخدمين (إنشاء، تعديل، حذف).
        مراقبة أداء النظام العام.
        تكوين إعدادات النظام ومعايير جودة البيانات.
        الوصول إلى سجلات النظام وتقارير التدقيق.
        إدارة التراخيص والصلاحيات للمستخدمين.
        
        **محلل البيانات 🔬:**
        تحميل وتنزيل مجموعات البيانات.
        بدء عمليات تحليل جودة البيانات.
        مراجعة تقارير جودة البيانات وتوصيات التصحيح.
        تطبيق التعديلات المقترحة على البيانات.
        جدولة عمليات تحليل البيانات الدورية.
        
        
        4. **مميزات المشروع:**
        معالجة تلقائية للبيانات 🤖: يقوم النظام بمعالجة البيانات بشكل تلقائي فور استقبالها، مما يقلل من التدخل اليدوي ويوفر الوقت والجهد.
        تقارير تفصيلية قابلة للتخصيص 📝: يوفر النظام تقارير شاملة عن جودة البيانات، يمكن تخصيصها لتناسب احتياجات المستخدمين المختلفة، مع إمكانية تصديرها بتنسيقات متعددة.
        إدارة صلاحيات دقيقة 🔑: يدعم النظام نظام صلاحيات متعدد المستويات يضمن وصول كل مستخدم إلى البيانات والوظائف المسموح بها فقط، مما يعزز أمان البيانات.
        تتبع حالة العمليات ⏳: يمكن للمستخدمين تتبع حالة عمليات تحليل ومعالجة البيانات في الوقت الفعلي، وتلقي إشعارات عند اكتمالها أو عند حدوث أخطاء.
        ميزة "علامة الجودة الرقمية" 🏅:
        الهدف منها: تهدف هذه الميزة إلى تزويد المستخدمين (خاصة صناع القرار) بمقياس مرئي وموثوق لجودة مجموعات البيانات التي يتم معالجتها داخل النظام. إنها بمثابة شهادة لجودة البيانات بعد إتمام عمليات التحليل والتصحيح.
        كيف تعمل داخل النظام: بعد اكتمال عملية تحليل وتصحيح مجموعة بيانات معينة، يقوم النظام بتقييم شامل لجودة البيانات بناءً على معايير محددة مسبقًا (مثل نسبة البيانات المكررة، نسبة البيانات المفقودة، مدى التناسق). بناءً على هذا التقييم، يتم منح "علامة جودة رقمية" لمجموعة البيانات، والتي يمكن أن تكون على شكل "ممتاز"، "جيد جدًا"، "مقبول"، أو "ضعيف"، مع عرض تفاصيل النقاط التي تم تقييمها.
        أثرها على المستخدم أو سمعة النظام: تزيد هذه الميزة من ثقة المستخدم في البيانات التي يتعامل معها، حيث يمكنهم الاعتماد على هذه العلامة كمؤشر سريع وموثوق لجودة البيانات. كما تعزز من سمعة النظام كمصدر موثوق به لتحسين جودة البيانات، مما يجذب المزيد من المستخدمين.
        سبب تميزها عن بقية الأنظمة المشابهة: تتميز هذه الميزة بكونها تقدم تقييمًا مرئيًا ومصنفًا لجودة البيانات، بدلاً من مجرد عرض تقارير تفصيلية تتطلب تحليلًا يدويًا. إنها توفر للمستخدم لمحة سريعة وشاملة عن حالة البيانات، وتساهم في بناء ثقة أكبر في النظام كأداة لتحسين جودة البيانات بشكل ملموس.
        
        5. **الهدف الوطني:**
        يدعم المشروع التوجه الوطني المتمثل في تعزيز الاستدامة الرقمية ♻️ عبر تحسين وتطوير قواعد البيانات، مما يسهم في تحقيق ركائز رؤية المملكة العربية السعودية 2030 🇸🇦 المتعلقة ببناء اقتصاد رقمي مزدهر ومجتمع معلوماتي معرفي.
        
        6. **شرح الخوارزميات:**
        تحليل الشكل العام (Pattern Recognition) 🔎:
        شرح علمي مبسط: هي عملية اكتشاف الأنماط أو العلاقات المتكررة في البيانات. تهدف إلى التعرف على تشابهات أو اختلافات ضمن مجموعات كبيرة من البيانات.
        الاستخدام داخل النظام: تُستخدم هذه الخوارزمية في "بيانات بلس" لاكتشاف الأنماط غير المتسقة في البيانات، مثل اختلاف تنسيقات التاريخ أو الأرقام، أو الأخطاء الإملائية الشائعة في الأسماء أو العناوين.
        مثال تطبيقي: إذا كانت قاعدة البيانات تحتوي على أسماء مدن مكتوبة بأكثر من شكل (مثلاً: "الرياض" و "رياض" و "Riyadh")، فإن تحليل الشكل العام سيكتشف هذه التباينات ويقترح توحيدها إلى شكل واحد، مما يحسن من جودة البيانات ويجعلها أكثر قابلية للتحليل.
        شجرة البحث الثنائية (Binary Search Tree - BST) 🌳:
        شرح علمي مبسط: هي هيكل بيانات شجري منظم، حيث تكون قيمة كل عقدة أكبر من جميع قيم العقد في فرعها الأيسر وأقل من جميع قيم العقد في فرعها الأيمن. هذا التنظيم يسهل عمليات البحث، الإضافة، والحذف بكفاءة عالية.
        الاستخدام داخل النظام: تُستخدم شجرة البحث الثنائية في "بيانات بلس" لتحسين كفاءة عمليات البحث عن البيانات المكررة أو المتشابهة. عند تحليل مجموعة بيانات كبيرة، يمكن تخزين العناصر في شجرة بحث ثنائية، مما يجعل عملية التحقق من وجود تكرارات أسرع بكثير.
        مثال تطبيقي: عند تنظيف قائمة بريدية طويلة جدًا من عناوين البريد الإلكتروني المكررة، يمكن إضافة كل عنوان بريد إلكتروني إلى شجرة بحث ثنائية. إذا حاول النظام إضافة عنوان موجود بالفعل، فسيتم اكتشافه بسرعة، مما يوفر وقتًا وجهدًا كبيرًا مقارنة بالبحث الخطي في القائمة بأكملها.
        
        7. **لغات البرمجة، التقنيات، وخوارزميات الذكاء الاصطناعي:**
        **لغات البرمجة 💻:**
        Swift: تُستخدم لتطوير تطبيق الجوال على منصة iOS لتقديم واجهة مستخدم سريعة وفعالة.
        Kotlin: تُستخدم لتطوير تطبيق الجوال على منصة Android لضمان التوافق والأداء الأمثل.
        Python: تُستخدم في الواجهة الخلفية لمعالجة البيانات وتطبيق الخوارزميات، وذلك لغناها بالمكتبات المتخصصة في تحليل البيانات.
       ** التقنيات المستخدمة 🛠️:**
        RESTful APIs: تُستخدم لتمكين الاتصال وتبادل البيانات بين تطبيق الجوال والواجهة الخلفية للنظام بشكل فعال وآمن.
        Docker: تُستخدم لحزم ونشر مكونات النظام كحاويات مستقلة، مما يضمن قابلية النقل وسهولة الإدارة.
        Git: يُستخدم لإدارة إصدارات الكود البرمجي والتعاون بين المطورين بشكل فعال.
       ** نوع قواعد البيانات 🗄️:**
        PostgreSQL: تُستخدم كقاعدة بيانات علائقية قوية وموثوقة لتخزين البيانات المنظمة وإدارة معلومات المستخدمين والعمليات.

        """
      else:
          # 'integration' is missing or empty, but 'allow_suggested_users' is present
        prompt = f"""
         الرجاء توليد نظام متكامل لمشروع تخرج مبتكر، يشمل جميع المكونات الأساسية لأي نظام احترافي مثل:
        • استقبال ومعالجة البيانات
        • تحليل أو تصنيف أو تنفيذ وظائف حسب المجال
        • إدارة العمليات وتتبع حالتها
        • إنشاء تقارير، إشعارات، وصلاحيات متعددة
        • تكامل مع أنظمة خارجية فقط إذا تم تحديد ذلك من قبل المستخدم
        ❗️ إذا كانت بعض الحقول غير محددة من قبل المستخدم، يُسمح للنظام باقتراح قيم مناسبة فقط لهذه الحقول المفقودة، بشرط أن تكون منطقية ومتناسبة مع نوع المشروع والمجال.
        ❌ لا يُسمح بتعديل أو استبدال أي قيمة تم تحديدها من قِبل المستخدم صراحة.
        ❌ لا يُسمح بتضمين أي تقنيات أو خوارزميات أو مفاهيم تتعلق بالذكاء الاصطناعي ما لم يتم طلبها صراحة ضمن بيانات المستخدم.
        ________________________________________
        ✅ تنسيق الرد المطلوب:
        1. **العنوان المقترح:**
        [عنوان واضح ومبتكر للفكرة]
        
        2. **وصف تفصيلي للفكرة مع أمثلة:**
        شرح شامل لطبيعة النظام، المشكلة التي يعالجها، طريقة استخدامه، الجهات المستفيدة، وسيناريوهات تطبيقية حقيقية.
        
        3. **المستخدمون ووظائفهم في النظام:**
        • يجب أن يشمل النظام "مدير النظام".
        • يدعم: تسجيل الدخول، تسجيل الخروج، إنشاء حساب جديد إن لزم.
        • لكل مستخدم: اذكر 5 وظائف واضحة على الأقل.
        o يجب توليد مستخدم واحد على الأقل فعليًا.
        o تحديد صفته، و5 وظائف له.
        • يُمنع ذكر اسماء نهائياً
        • يُمنع تكرار المستخدمين أو وظائفهم في قسم منفصل.
        
        4. **مميزات المشروع (لا تقل عن 5):**
        • مميزات عملية توضح قوة المشروع وجدواه.
        • يجب أن يحتوي النظام على ميزة مبتكرة فارقة تميّزه عن الأنظمة الأخرى المشابهة (سواء كان موقعًا أو تطبيقًا).
        • يُفضل أن تكون هذه الميزة مرتبطة بـ "علامة جودة" أو وظيفة حصرية تضيف قيمة واضحة.
        • يجب شرحها من حيث:
        o الهدف منها
        o كيف تعمل داخل النظام
        o أثرها على المستخدم أو سمعة النظام
        o سبب تميزها عن بقية الأنظمة المشابهة
        
        5. **الهدف الوطني:**
        • شرح مختصر لا يتجاوز سطرين فقط.
        • يُولد بناءً على فكرة المشروع الفعلية والمجال المستخدم.
        • يوضّح كيف يدعم المشروع فعليًا التوجه الوطني الذي حدده المستخدم (مثل: دعم التعليم الرقمي، تعزيز نمط الحياة الصحي...).
        • يجب ربطه بـ رؤية المملكة العربية السعودية 2030.
        • يُمنع تثبيت أو نسخ هدف عام لا يعكس الفكرة الفعلية.
        
        6. **شرح الخوارزميات:**
        • شرح علمي مبسط لكل خوارزمية مستخدمة (مثل: Merge Sort، K-Means).
        • توضيح كيف تُستخدم فعليًا داخل النظام.
        • إرفاق مثال تطبيقي لبيان فائدتها.
        
        7. **لغات البرمجة، التقنيات، وخوارزميات الذكاء الاصطناعي:**
        • لكل لغة أو تقنية: شرح لا يتجاوز سطرًا واحدًا يوضح وظيفتها داخل المشروع.
        • في حال تم استخدام الذكاء الاصطناعي:
        o تحديد نوع الذكاء (تحليل، تصنيف، تنبؤ...)
        o ما الذي يتم تحليله أو تصنيفه داخل النظام
        o شرح مبسط لطريقة عمل الخوارزمية
        o مثال تطبيقي يوضح فائدة الخوارزمية
        
        ________________________________________
        📌 القيود الصارمة (يجب الالتزام بها):
        • ✅ توليد نظام متكامل وظيفيًا، لا مجرد فكرة أو أداة جزئية.
        • ✅ الالتزام التام بجميع الأدوات، اللغات، والتقنيات المذكورة فقط.
        • ✅ لا يُسمح بتعديل أو تجاوز أي قيمة صرّح بها المستخدم.
        • ✅ يُسمح باقتراح القيم فقط في حال غيابها.
        • ✅ يُمنع تضمين الذكاء الاصطناعي نهائيًا ما لم يُطلب صراحة.
        • ✅ لكل مستخدم: 5 وظائف على الأقل.
        • ✅ عند السماح بتوليد مستخدمين إضافيين: يجب توليد مستخدم فعلي مع وظائفه.
        • ✅ لا يُسمح بتكرار عرض المستخدمين أو وظائفهم.
        • ✅ شرح اللغات لا يتجاوز سطرًا واحدًا.
        • ✅ الهدف الوطني لا يتجاوز سطرين، ويُولّد حسب الفكرة ويرتبط برؤية المملكة 2030.
        • ✅ يجب أن يحتوي المشروع على ميزة فارقة تميز النظام بوضوح عن الأنظمة الأخرى.
        • ✅ عدم وضع اي جمل افتتاحية او جمل ختامية في الرد النهائي
        • ✅ الالتزام بترقيم كل الفقرات والعناوين الرئيسية والعناوين الفرعية والاقسام الداخلية يتم ترقيمها بطريقة منسقة
        • ✅ يرجى تنسيق النص واضافة الايموجز والعناوين الرئيسية والفرعية بشكل منسق ع طريقة Markdown
        • ✅ يجب التاكد ان العناويين الرئيسية بحجم وخط منسقين يظهرون انهم عناويين رئيسية كذلك العناوين الفرعية والترقيم
        * ✅ برجاء عدم وضع جمل افتتاحية او ختامية يجب الالتزام بالتركيز ع توليد الفكرة مباشرة
        * ✅ يجب توليد قواعد البيانات المقترحة و التقنيات المستخدمة اذا كانت هذه القيم فارغة او لم يتم عرضها من المستخدم "technologies": "","database_type": ""
        * ✅ يجب استخدام الفكرة الاولية او initial_idea اللتي ارسلها المستخدم في توليد فكرة المشروع )مهمة جدا(
        _______________________________________
        هذه البيانات اللتي يتم بناءا عليها بناء الفكرة بكل تفاصيلها. الرجاء الالتزام بهذه المعطيات فقط
        ## 📂 البيانات المتوفرة:
        تفاصيل الفكرة :

        نوع الدراسة: {data.get("study_type", "")}
        التخصص: {data.get("specialization", "")}
        المجال: {data.get("field", "")}
        التوجه الوطني: {data.get("national_direction", "")}
        خوارزميات المشروع: {data.get("project_algorithms", "")}
        الفئة المستهدفة: {data.get("target_audience", "")}
        نوع النظام: {data.get("system_type", "")}
        لغات البرمجة: {data.get("programming_languages", "")}
        التقنيات المستخدمة: {data.get("technologies", "")}
        نوع قواعد البيانات: {data.get("database_type", "")}
        نوع البيانات المستخدمة في الذكاء الاصطناعي: {data.get("ai_data_type", "")}
        أنواع الاستخدام للذكاء الاصطناعي: {data.get("ai_use_types", "")}
        الخوارزميات المقترحة للذكاء الاصطناعي: {data.get("ai_algorithms", "")}
        ملاحظات المستخدم أو وصفه الأولي: {data.get("initial_idea", "")}
         توليد مستخدمين اخرين غير الفئة المستهدفة المذكورة في نطاق فكرة المشروع : {data.get("allow_suggested_users", "")}

        هذا مثال حي يمكنك الاستفادة منه
        1. **العنوان المقترح:**
        نظام "بيانات بلس" لتحسين جودة قواعد البيانات 🌟
        
        2. **وصف تفصيلي للفكرة مع أمثلة:**
        نظام "بيانات بلس" 📊 هو تطبيق جوال مبتكر صُمم خصيصًا لمعالجة المشكلات الشائعة في جودة قواعد البيانات، مثل التكرار، البيانات غير المتسقة، والأخطاء الإملائية. هذه المشكلات تؤثر سلبًا على موثوقية البيانات وصحة القرارات المتخذة بناءً عليها. يعمل النظام عن طريق استقبال البيانات، معالجتها، ثم تحليلها لاكتشاف هذه المشاكل، وتقديم حلول مقترحة لتحسينها.
        يستطيع محللو البيانات وصناع القرار استخدام النظام لتحليل مجموعات بيانات مختلفة، مثل بيانات العملاء 🧑‍🤝‍🧑 أو المنتجات 🛍️، وتصحيح الأخطاء بشكل فعال لضمان دقة المعلومات.
        مثال 1: يمكن لشركة تجزئة 🏪 استخدام "بيانات بلس" لتنظيف قاعدة بيانات عملائها من الإدخالات المكررة أو غير الدقيقة، مما يتيح لهم استهداف العملاء بشكل أفضل وتحسين حملاتهم التسويقية 📈.
        مثال 2: سيناريو آخر هو استخدامه من قبل مؤسسة مالية 💰 للتحقق من سلامة بيانات المعاملات، وتقليل الأخطاء التي قد تؤدي إلى خسائر مالية أو عدم امتثال للوائح.
        
        3. **المستخدمون ووظائفهم في النظام:**
        **مدير النظام ⚙️:**
        إدارة حسابات المستخدمين (إنشاء، تعديل، حذف).
        مراقبة أداء النظام العام.
        تكوين إعدادات النظام ومعايير جودة البيانات.
        الوصول إلى سجلات النظام وتقارير التدقيق.
        إدارة التراخيص والصلاحيات للمستخدمين.
        
        **محلل البيانات 🔬:**
        تحميل وتنزيل مجموعات البيانات.
        بدء عمليات تحليل جودة البيانات.
        مراجعة تقارير جودة البيانات وتوصيات التصحيح.
        تطبيق التعديلات المقترحة على البيانات.
        جدولة عمليات تحليل البيانات الدورية.
        
        **صانع القرار 🎯:**
        عرض لوحات المعلومات المخصصة لجودة البيانات.
        مراجعة التقارير الموجزة عن سلامة البيانات.
        تقييم تأثير جودة البيانات على مؤشرات الأداء الرئيسية.
        طلب تحليلات إضافية لجودة البيانات.
        تلقي إشعارات حول التغييرات الهامة في جودة البيانات.
        
        4. **مميزات المشروع:**
        معالجة تلقائية للبيانات 🤖: يقوم النظام بمعالجة البيانات بشكل تلقائي فور استقبالها، مما يقلل من التدخل اليدوي ويوفر الوقت والجهد.
        تقارير تفصيلية قابلة للتخصيص 📝: يوفر النظام تقارير شاملة عن جودة البيانات، يمكن تخصيصها لتناسب احتياجات المستخدمين المختلفة، مع إمكانية تصديرها بتنسيقات متعددة.
        إدارة صلاحيات دقيقة 🔑: يدعم النظام نظام صلاحيات متعدد المستويات يضمن وصول كل مستخدم إلى البيانات والوظائف المسموح بها فقط، مما يعزز أمان البيانات.
        تتبع حالة العمليات ⏳: يمكن للمستخدمين تتبع حالة عمليات تحليل ومعالجة البيانات في الوقت الفعلي، وتلقي إشعارات عند اكتمالها أو عند حدوث أخطاء.
        ميزة "علامة الجودة الرقمية" 🏅:
        الهدف منها: تهدف هذه الميزة إلى تزويد المستخدمين (خاصة صناع القرار) بمقياس مرئي وموثوق لجودة مجموعات البيانات التي يتم معالجتها داخل النظام. إنها بمثابة شهادة لجودة البيانات بعد إتمام عمليات التحليل والتصحيح.
        كيف تعمل داخل النظام: بعد اكتمال عملية تحليل وتصحيح مجموعة بيانات معينة، يقوم النظام بتقييم شامل لجودة البيانات بناءً على معايير محددة مسبقًا (مثل نسبة البيانات المكررة، نسبة البيانات المفقودة، مدى التناسق). بناءً على هذا التقييم، يتم منح "علامة جودة رقمية" لمجموعة البيانات، والتي يمكن أن تكون على شكل "ممتاز"، "جيد جدًا"، "مقبول"، أو "ضعيف"، مع عرض تفاصيل النقاط التي تم تقييمها.
        أثرها على المستخدم أو سمعة النظام: تزيد هذه الميزة من ثقة المستخدم في البيانات التي يتعامل معها، حيث يمكنهم الاعتماد على هذه العلامة كمؤشر سريع وموثوق لجودة البيانات. كما تعزز من سمعة النظام كمصدر موثوق به لتحسين جودة البيانات، مما يجذب المزيد من المستخدمين.
        سبب تميزها عن بقية الأنظمة المشابهة: تتميز هذه الميزة بكونها تقدم تقييمًا مرئيًا ومصنفًا لجودة البيانات، بدلاً من مجرد عرض تقارير تفصيلية تتطلب تحليلًا يدويًا. إنها توفر للمستخدم لمحة سريعة وشاملة عن حالة البيانات، وتساهم في بناء ثقة أكبر في النظام كأداة لتحسين جودة البيانات بشكل ملموس.
        
        5. **الهدف الوطني:**
        يدعم المشروع التوجه الوطني المتمثل في تعزيز الاستدامة الرقمية ♻️ عبر تحسين وتطوير قواعد البيانات، مما يسهم في تحقيق ركائز رؤية المملكة العربية السعودية 2030 🇸🇦 المتعلقة ببناء اقتصاد رقمي مزدهر ومجتمع معلوماتي معرفي.
        
        6. **شرح الخوارزميات:**
        تحليل الشكل العام (Pattern Recognition) 🔎:
        شرح علمي مبسط: هي عملية اكتشاف الأنماط أو العلاقات المتكررة في البيانات. تهدف إلى التعرف على تشابهات أو اختلافات ضمن مجموعات كبيرة من البيانات.
        الاستخدام داخل النظام: تُستخدم هذه الخوارزمية في "بيانات بلس" لاكتشاف الأنماط غير المتسقة في البيانات، مثل اختلاف تنسيقات التاريخ أو الأرقام، أو الأخطاء الإملائية الشائعة في الأسماء أو العناوين.
        مثال تطبيقي: إذا كانت قاعدة البيانات تحتوي على أسماء مدن مكتوبة بأكثر من شكل (مثلاً: "الرياض" و "رياض" و "Riyadh")، فإن تحليل الشكل العام سيكتشف هذه التباينات ويقترح توحيدها إلى شكل واحد، مما يحسن من جودة البيانات ويجعلها أكثر قابلية للتحليل.
        شجرة البحث الثنائية (Binary Search Tree - BST) 🌳:
        شرح علمي مبسط: هي هيكل بيانات شجري منظم، حيث تكون قيمة كل عقدة أكبر من جميع قيم العقد في فرعها الأيسر وأقل من جميع قيم العقد في فرعها الأيمن. هذا التنظيم يسهل عمليات البحث، الإضافة، والحذف بكفاءة عالية.
        الاستخدام داخل النظام: تُستخدم شجرة البحث الثنائية في "بيانات بلس" لتحسين كفاءة عمليات البحث عن البيانات المكررة أو المتشابهة. عند تحليل مجموعة بيانات كبيرة، يمكن تخزين العناصر في شجرة بحث ثنائية، مما يجعل عملية التحقق من وجود تكرارات أسرع بكثير.
        مثال تطبيقي: عند تنظيف قائمة بريدية طويلة جدًا من عناوين البريد الإلكتروني المكررة، يمكن إضافة كل عنوان بريد إلكتروني إلى شجرة بحث ثنائية. إذا حاول النظام إضافة عنوان موجود بالفعل، فسيتم اكتشافه بسرعة، مما يوفر وقتًا وجهدًا كبيرًا مقارنة بالبحث الخطي في القائمة بأكملها.
        
        7. **لغات البرمجة، التقنيات، وخوارزميات الذكاء الاصطناعي:**
        **لغات البرمجة 💻:**
        Swift: تُستخدم لتطوير تطبيق الجوال على منصة iOS لتقديم واجهة مستخدم سريعة وفعالة.
        Kotlin: تُستخدم لتطوير تطبيق الجوال على منصة Android لضمان التوافق والأداء الأمثل.
        Python: تُستخدم في الواجهة الخلفية لمعالجة البيانات وتطبيق الخوارزميات، وذلك لغناها بالمكتبات المتخصصة في تحليل البيانات.
       ** التقنيات المستخدمة 🛠️:**
        RESTful APIs: تُستخدم لتمكين الاتصال وتبادل البيانات بين تطبيق الجوال والواجهة الخلفية للنظام بشكل فعال وآمن.
        Docker: تُستخدم لحزم ونشر مكونات النظام كحاويات مستقلة، مما يضمن قابلية النقل وسهولة الإدارة.
        Git: يُستخدم لإدارة إصدارات الكود البرمجي والتعاون بين المطورين بشكل فعال.
       ** نوع قواعد البيانات 🗄️:**
        PostgreSQL: تُستخدم كقاعدة بيانات علائقية قوية وموثوقة لتخزين البيانات المنظمة وإدارة معلومات المستخدمين والعمليات.

        
        """
    else:
      if 'allow_suggested_users' not in data or not data['allow_suggested_users']:
            # 'integration' is present, but 'allow_suggested_users' is missing or empty
            prompt = f"""
         الرجاء توليد نظام متكامل لمشروع تخرج مبتكر، يشمل جميع المكونات الأساسية لأي نظام احترافي مثل:
        • استقبال ومعالجة البيانات
        • تحليل أو تصنيف أو تنفيذ وظائف حسب المجال
        • إدارة العمليات وتتبع حالتها
        • إنشاء تقارير، إشعارات، وصلاحيات متعددة
        • تكامل مع أنظمة خارجية فقط إذا تم تحديد ذلك من قبل المستخدم
        ❗️ إذا كانت بعض الحقول غير محددة من قبل المستخدم، يُسمح للنظام باقتراح قيم مناسبة فقط لهذه الحقول المفقودة، بشرط أن تكون منطقية ومتناسبة مع نوع المشروع والمجال.
        ❌ لا يُسمح بتعديل أو استبدال أي قيمة تم تحديدها من قِبل المستخدم صراحة.
        ❌ لا يُسمح بتضمين أي تقنيات أو خوارزميات أو مفاهيم تتعلق بالذكاء الاصطناعي ما لم يتم طلبها صراحة ضمن بيانات المستخدم.
        ________________________________________
        ✅ تنسيق الرد المطلوب:
        1. **العنوان المقترح:**
        [عنوان واضح ومبتكر للفكرة]
        
        2. **وصف تفصيلي للفكرة مع أمثلة:**
        شرح شامل لطبيعة النظام، المشكلة التي يعالجها، طريقة استخدامه، الجهات المستفيدة، وسيناريوهات تطبيقية حقيقية.
        
        3. **المستخدمون ووظائفهم في النظام:**
        • يجب أن يشمل النظام "مدير النظام".
        • يدعم: تسجيل الدخول، تسجيل الخروج، إنشاء حساب جديد إن لزم.
        • لكل مستخدم: اذكر 5 وظائف واضحة على الأقل.
        o تحديد صفته، و5 وظائف له.
        • يُمنع ذكر اسماء نهائياً
        • يُمنع تكرار المستخدمين أو وظائفهم في قسم منفصل.
        
        4. **مميزات المشروع (لا تقل عن 5):**
        • مميزات عملية توضح قوة المشروع وجدواه.
        • يجب أن يحتوي النظام على ميزة مبتكرة فارقة تميّزه عن الأنظمة الأخرى المشابهة (سواء كان موقعًا أو تطبيقًا).
        • يُفضل أن تكون هذه الميزة مرتبطة بـ "علامة جودة" أو وظيفة حصرية تضيف قيمة واضحة.
        • يجب شرحها من حيث:
        o الهدف منها
        o كيف تعمل داخل النظام
        o أثرها على المستخدم أو سمعة النظام
        o سبب تميزها عن بقية الأنظمة المشابهة
        
        5. **الهدف الوطني:**
        • شرح مختصر لا يتجاوز سطرين فقط.
        • يُولد بناءً على فكرة المشروع الفعلية والمجال المستخدم.
        • يوضّح كيف يدعم المشروع فعليًا التوجه الوطني الذي حدده المستخدم (مثل: دعم التعليم الرقمي، تعزيز نمط الحياة الصحي...).
        • يجب ربطه بـ رؤية المملكة العربية السعودية 2030.
        • يُمنع تثبيت أو نسخ هدف عام لا يعكس الفكرة الفعلية.
        
        6. **شرح الخوارزميات:**
        • شرح علمي مبسط لكل خوارزمية مستخدمة (مثل: Merge Sort، K-Means).
        • توضيح كيف تُستخدم فعليًا داخل النظام.
        • إرفاق مثال تطبيقي لبيان فائدتها.
        
        7. **لغات البرمجة، التقنيات، وخوارزميات الذكاء الاصطناعي:**
        • لكل لغة أو تقنية: شرح لا يتجاوز سطرًا واحدًا يوضح وظيفتها داخل المشروع.
        • في حال تم استخدام الذكاء الاصطناعي:
        o تحديد نوع الذكاء (تحليل، تصنيف، تنبؤ...)
        o ما الذي يتم تحليله أو تصنيفه داخل النظام
        o شرح مبسط لطريقة عمل الخوارزمية
        o مثال تطبيقي يوضح فائدة الخوارزمية
        
        8. **التكامل مع الأنظمة الأخرى:**
        • إذا لم يتم تحديد نوع التكامل من قبل المستخدم، يُتجاهل هذا القسم تمامًا.
        • إذا تم تحديد نوع التكامل (مثل: API خارجي، ذكاء اصطناعي...):
        o شرح وظيفة التكامل عمليًا (مثل: تبادل بيانات، مزامنة حسابات، استخدام نماذج خارجية).
        o تحديد مكان حدوث التكامل داخل النظام.
        o شرح القيمة المضافة من هذا التكامل.
        ________________________________________
        📌 القيود الصارمة (يجب الالتزام بها):
        • ✅ توليد نظام متكامل وظيفيًا، لا مجرد فكرة أو أداة جزئية.
        • ✅ الالتزام التام بجميع الأدوات، اللغات، والتقنيات المذكورة فقط.
        • ✅ لا يُسمح بتعديل أو تجاوز أي قيمة صرّح بها المستخدم.
        • ✅ يُسمح باقتراح القيم فقط في حال غيابها.
        • ✅ يُمنع تضمين الذكاء الاصطناعي نهائيًا ما لم يُطلب صراحة.
        • ✅ لكل مستخدم: 5 وظائف على الأقل.
        • ✅ عند السماح بتوليد مستخدمين إضافيين: يجب توليد مستخدم فعلي مع وظائفه.
        • ✅ لا يُسمح بتكرار عرض المستخدمين أو وظائفهم.
        • ✅ شرح اللغات لا يتجاوز سطرًا واحدًا.
        • ✅ الهدف الوطني لا يتجاوز سطرين، ويُولّد حسب الفكرة ويرتبط برؤية المملكة 2030.
        • ✅ يجب أن يحتوي المشروع على ميزة فارقة تميز النظام بوضوح عن الأنظمة الأخرى.
        • ✅ عدم وضع اي جمل افتتاحية او جمل ختامية في الرد النهائي
        • ✅ الالتزام بترقيم كل الفقرات والعناوين الرئيسية والعناوين الفرعية والاقسام الداخلية يتم ترقيمها بطريقة منسقة
        • ✅ في حالة ان المتغير integration فارغ او لم يتم ارساله يتم حذف الفقرة رقم 8 من الاجابة نهائيا وهي التكامل مع الانظمة الاخرى وعدم ذكرها
        • ✅ يرجى تنسيق النص واضافة الايموجز والعناوين الرئيسية والفرعية بشكل منسق ع طريقة Markdown
        • ✅ يجب التاكد ان العناويين الرئيسية بحجم وخط منسقين يظهرون انهم عناويين رئيسية كذلك العناوين الفرعية والترقيم
        * ✅ هذا مثال توضيحي للشكل النهائي للرد المنتظر منك مع العلم ان فقرة 8 التكامل مع الانظمة الاخرى يتم اضافتها اذا تم ذكر ذلك ويتم حذفها اذا كانت قيمة integration فارغة او غير موجودة
        * ✅ برجاء عدم وضع جمل افتتاحية او ختامية يجب الالتزام بالتركيز ع توليد الفكرة مباشرة
        * ✅ يجب توليد قواعد البيانات المقترحة و التقنيات المستخدمة اذا كانت هذه القيم فارغة او لم يتم عرضها من المستخدم "technologies": "","database_type": ""
        * ✅ يجب استخدام الفكرة الاولية او initial_idea اللتي ارسلها المستخدم في توليد فكرة المشروع )مهمة جدا(
        _______________________________________
        هذه البيانات اللتي يتم بناءا عليها بناء الفكرة بكل تفاصيلها. الرجاء الالتزام بهذه المعطيات فقط
        ## 📂 البيانات المتوفرة:
        تفاصيل الفكرة :

        نوع الدراسة: {data.get("study_type", "")}
        التخصص: {data.get("specialization", "")}
        المجال: {data.get("field", "")}
        التوجه الوطني: {data.get("national_direction", "")}
        خوارزميات المشروع: {data.get("project_algorithms", "")}
        الفئة المستهدفة: {data.get("target_audience", "")}
        نوع النظام: {data.get("system_type", "")}
        التكامل مع أنظمة أخرى: {data.get("integration", "")}
        لغات البرمجة: {data.get("programming_languages", "")}
        التقنيات المستخدمة: {data.get("technologies", "")}
        نوع قواعد البيانات: {data.get("database_type", "")}
        نوع البيانات المستخدمة في الذكاء الاصطناعي: {data.get("ai_data_type", "")}
        أنواع الاستخدام للذكاء الاصطناعي: {data.get("ai_use_types", "")}
        الخوارزميات المقترحة للذكاء الاصطناعي: {data.get("ai_algorithms", "")}
        ملاحظات المستخدم أو وصفه الأولي: {data.get("initial_idea", "")}

        هذا مثال حي يمكنك الاستفادة منه
        1. **العنوان المقترح:**
        نظام "بيانات بلس" لتحسين جودة قواعد البيانات 🌟
        
        2. **وصف تفصيلي للفكرة مع أمثلة:**
        نظام "بيانات بلس" 📊 هو تطبيق جوال مبتكر صُمم خصيصًا لمعالجة المشكلات الشائعة في جودة قواعد البيانات، مثل التكرار، البيانات غير المتسقة، والأخطاء الإملائية. هذه المشكلات تؤثر سلبًا على موثوقية البيانات وصحة القرارات المتخذة بناءً عليها. يعمل النظام عن طريق استقبال البيانات، معالجتها، ثم تحليلها لاكتشاف هذه المشاكل، وتقديم حلول مقترحة لتحسينها.
        يستطيع محللو البيانات وصناع القرار استخدام النظام لتحليل مجموعات بيانات مختلفة، مثل بيانات العملاء 🧑‍🤝‍🧑 أو المنتجات 🛍️، وتصحيح الأخطاء بشكل فعال لضمان دقة المعلومات.
        مثال 1: يمكن لشركة تجزئة 🏪 استخدام "بيانات بلس" لتنظيف قاعدة بيانات عملائها من الإدخالات المكررة أو غير الدقيقة، مما يتيح لهم استهداف العملاء بشكل أفضل وتحسين حملاتهم التسويقية 📈.
        مثال 2: سيناريو آخر هو استخدامه من قبل مؤسسة مالية 💰 للتحقق من سلامة بيانات المعاملات، وتقليل الأخطاء التي قد تؤدي إلى خسائر مالية أو عدم امتثال للوائح.
        
        3. **المستخدمون ووظائفهم في النظام:**
        **مدير النظام ⚙️:**
        إدارة حسابات المستخدمين (إنشاء، تعديل، حذف).
        مراقبة أداء النظام العام.
        تكوين إعدادات النظام ومعايير جودة البيانات.
        الوصول إلى سجلات النظام وتقارير التدقيق.
        إدارة التراخيص والصلاحيات للمستخدمين.
        
        **محلل البيانات 🔬:**
        تحميل وتنزيل مجموعات البيانات.
        بدء عمليات تحليل جودة البيانات.
        مراجعة تقارير جودة البيانات وتوصيات التصحيح.
        تطبيق التعديلات المقترحة على البيانات.
        جدولة عمليات تحليل البيانات الدورية.
        
        **صانع القرار 🎯:**
        عرض لوحات المعلومات المخصصة لجودة البيانات.
        مراجعة التقارير الموجزة عن سلامة البيانات.
        تقييم تأثير جودة البيانات على مؤشرات الأداء الرئيسية.
        طلب تحليلات إضافية لجودة البيانات.
        تلقي إشعارات حول التغييرات الهامة في جودة البيانات.
        
        4. **مميزات المشروع:**
        معالجة تلقائية للبيانات 🤖: يقوم النظام بمعالجة البيانات بشكل تلقائي فور استقبالها، مما يقلل من التدخل اليدوي ويوفر الوقت والجهد.
        تقارير تفصيلية قابلة للتخصيص 📝: يوفر النظام تقارير شاملة عن جودة البيانات، يمكن تخصيصها لتناسب احتياجات المستخدمين المختلفة، مع إمكانية تصديرها بتنسيقات متعددة.
        إدارة صلاحيات دقيقة 🔑: يدعم النظام نظام صلاحيات متعدد المستويات يضمن وصول كل مستخدم إلى البيانات والوظائف المسموح بها فقط، مما يعزز أمان البيانات.
        تتبع حالة العمليات ⏳: يمكن للمستخدمين تتبع حالة عمليات تحليل ومعالجة البيانات في الوقت الفعلي، وتلقي إشعارات عند اكتمالها أو عند حدوث أخطاء.
        ميزة "علامة الجودة الرقمية" 🏅:
        الهدف منها: تهدف هذه الميزة إلى تزويد المستخدمين (خاصة صناع القرار) بمقياس مرئي وموثوق لجودة مجموعات البيانات التي يتم معالجتها داخل النظام. إنها بمثابة شهادة لجودة البيانات بعد إتمام عمليات التحليل والتصحيح.
        كيف تعمل داخل النظام: بعد اكتمال عملية تحليل وتصحيح مجموعة بيانات معينة، يقوم النظام بتقييم شامل لجودة البيانات بناءً على معايير محددة مسبقًا (مثل نسبة البيانات المكررة، نسبة البيانات المفقودة، مدى التناسق). بناءً على هذا التقييم، يتم منح "علامة جودة رقمية" لمجموعة البيانات، والتي يمكن أن تكون على شكل "ممتاز"، "جيد جدًا"، "مقبول"، أو "ضعيف"، مع عرض تفاصيل النقاط التي تم تقييمها.
        أثرها على المستخدم أو سمعة النظام: تزيد هذه الميزة من ثقة المستخدم في البيانات التي يتعامل معها، حيث يمكنهم الاعتماد على هذه العلامة كمؤشر سريع وموثوق لجودة البيانات. كما تعزز من سمعة النظام كمصدر موثوق به لتحسين جودة البيانات، مما يجذب المزيد من المستخدمين.
        سبب تميزها عن بقية الأنظمة المشابهة: تتميز هذه الميزة بكونها تقدم تقييمًا مرئيًا ومصنفًا لجودة البيانات، بدلاً من مجرد عرض تقارير تفصيلية تتطلب تحليلًا يدويًا. إنها توفر للمستخدم لمحة سريعة وشاملة عن حالة البيانات، وتساهم في بناء ثقة أكبر في النظام كأداة لتحسين جودة البيانات بشكل ملموس.
        
        5. **الهدف الوطني:**
        يدعم المشروع التوجه الوطني المتمثل في تعزيز الاستدامة الرقمية ♻️ عبر تحسين وتطوير قواعد البيانات، مما يسهم في تحقيق ركائز رؤية المملكة العربية السعودية 2030 🇸🇦 المتعلقة ببناء اقتصاد رقمي مزدهر ومجتمع معلوماتي معرفي.
        
        6. **شرح الخوارزميات:**
        تحليل الشكل العام (Pattern Recognition) 🔎:
        شرح علمي مبسط: هي عملية اكتشاف الأنماط أو العلاقات المتكررة في البيانات. تهدف إلى التعرف على تشابهات أو اختلافات ضمن مجموعات كبيرة من البيانات.
        الاستخدام داخل النظام: تُستخدم هذه الخوارزمية في "بيانات بلس" لاكتشاف الأنماط غير المتسقة في البيانات، مثل اختلاف تنسيقات التاريخ أو الأرقام، أو الأخطاء الإملائية الشائعة في الأسماء أو العناوين.
        مثال تطبيقي: إذا كانت قاعدة البيانات تحتوي على أسماء مدن مكتوبة بأكثر من شكل (مثلاً: "الرياض" و "رياض" و "Riyadh")، فإن تحليل الشكل العام سيكتشف هذه التباينات ويقترح توحيدها إلى شكل واحد، مما يحسن من جودة البيانات ويجعلها أكثر قابلية للتحليل.
        شجرة البحث الثنائية (Binary Search Tree - BST) 🌳:
        شرح علمي مبسط: هي هيكل بيانات شجري منظم، حيث تكون قيمة كل عقدة أكبر من جميع قيم العقد في فرعها الأيسر وأقل من جميع قيم العقد في فرعها الأيمن. هذا التنظيم يسهل عمليات البحث، الإضافة، والحذف بكفاءة عالية.
        الاستخدام داخل النظام: تُستخدم شجرة البحث الثنائية في "بيانات بلس" لتحسين كفاءة عمليات البحث عن البيانات المكررة أو المتشابهة. عند تحليل مجموعة بيانات كبيرة، يمكن تخزين العناصر في شجرة بحث ثنائية، مما يجعل عملية التحقق من وجود تكرارات أسرع بكثير.
        مثال تطبيقي: عند تنظيف قائمة بريدية طويلة جدًا من عناوين البريد الإلكتروني المكررة، يمكن إضافة كل عنوان بريد إلكتروني إلى شجرة بحث ثنائية. إذا حاول النظام إضافة عنوان موجود بالفعل، فسيتم اكتشافه بسرعة، مما يوفر وقتًا وجهدًا كبيرًا مقارنة بالبحث الخطي في القائمة بأكملها.
        
        7. **لغات البرمجة، التقنيات، وخوارزميات الذكاء الاصطناعي:**
        **لغات البرمجة 💻:**
        Swift: تُستخدم لتطوير تطبيق الجوال على منصة iOS لتقديم واجهة مستخدم سريعة وفعالة.
        Kotlin: تُستخدم لتطوير تطبيق الجوال على منصة Android لضمان التوافق والأداء الأمثل.
        Python: تُستخدم في الواجهة الخلفية لمعالجة البيانات وتطبيق الخوارزميات، وذلك لغناها بالمكتبات المتخصصة في تحليل البيانات.
       ** التقنيات المستخدمة 🛠️:**
        RESTful APIs: تُستخدم لتمكين الاتصال وتبادل البيانات بين تطبيق الجوال والواجهة الخلفية للنظام بشكل فعال وآمن.
        Docker: تُستخدم لحزم ونشر مكونات النظام كحاويات مستقلة، مما يضمن قابلية النقل وسهولة الإدارة.
        Git: يُستخدم لإدارة إصدارات الكود البرمجي والتعاون بين المطورين بشكل فعال.
       ** نوع قواعد البيانات 🗄️:**
        PostgreSQL: تُستخدم كقاعدة بيانات علائقية قوية وموثوقة لتخزين البيانات المنظمة وإدارة معلومات المستخدمين والعمليات.

        8. **التكامل مع الأنظمة الأخرى:**
        • إذا لم يتم تحديد نوع التكامل من قبل المستخدم، يُتجاهل هذا القسم تمامًا.
        • إذا تم تحديد نوع التكامل (مثل: API خارجي، ذكاء اصطناعي...):
        o شرح وظيفة التكامل عمليًا (مثل: تبادل بيانات، مزامنة حسابات، استخدام نماذج خارجية).
        o تحديد مكان حدوث التكامل داخل النظام.
        o شرح القيمة المضافة من هذا التكامل.

        """
      else:
        # Both 'integration' and 'allow_suggested_users' are present    
        prompt = f"""
         الرجاء توليد نظام متكامل لمشروع تخرج مبتكر، يشمل جميع المكونات الأساسية لأي نظام احترافي مثل:
        • استقبال ومعالجة البيانات
        • تحليل أو تصنيف أو تنفيذ وظائف حسب المجال
        • إدارة العمليات وتتبع حالتها
        • إنشاء تقارير، إشعارات، وصلاحيات متعددة
        • تكامل مع أنظمة خارجية فقط إذا تم تحديد ذلك من قبل المستخدم
        ❗️ إذا كانت بعض الحقول غير محددة من قبل المستخدم، يُسمح للنظام باقتراح قيم مناسبة فقط لهذه الحقول المفقودة، بشرط أن تكون منطقية ومتناسبة مع نوع المشروع والمجال.
        ❌ لا يُسمح بتعديل أو استبدال أي قيمة تم تحديدها من قِبل المستخدم صراحة.
        ❌ لا يُسمح بتضمين أي تقنيات أو خوارزميات أو مفاهيم تتعلق بالذكاء الاصطناعي ما لم يتم طلبها صراحة ضمن بيانات المستخدم.
        ________________________________________
        ✅ تنسيق الرد المطلوب:
        1. **العنوان المقترح:**
        [عنوان واضح ومبتكر للفكرة]
        
        2. **وصف تفصيلي للفكرة مع أمثلة:**
        شرح شامل لطبيعة النظام، المشكلة التي يعالجها، طريقة استخدامه، الجهات المستفيدة، وسيناريوهات تطبيقية حقيقية.
        
        3. **المستخدمون ووظائفهم في النظام:**
        • يجب أن يشمل النظام "مدير النظام".
        • يدعم: تسجيل الدخول، تسجيل الخروج، إنشاء حساب جديد إن لزم.
        • لكل مستخدم: اذكر 5 وظائف واضحة على الأقل.
        o يجب توليد مستخدم واحد على الأقل فعليًا.
        o تحديد صفته، و5 وظائف له.
        • يُمنع ذكر اسماء نهائياً
        • يُمنع تكرار المستخدمين أو وظائفهم في قسم منفصل.
        
        4. **مميزات المشروع (لا تقل عن 5):**
        • مميزات عملية توضح قوة المشروع وجدواه.
        • يجب أن يحتوي النظام على ميزة مبتكرة فارقة تميّزه عن الأنظمة الأخرى المشابهة (سواء كان موقعًا أو تطبيقًا).
        • يُفضل أن تكون هذه الميزة مرتبطة بـ "علامة جودة" أو وظيفة حصرية تضيف قيمة واضحة.
        • يجب شرحها من حيث:
        o الهدف منها
        o كيف تعمل داخل النظام
        o أثرها على المستخدم أو سمعة النظام
        o سبب تميزها عن بقية الأنظمة المشابهة
        
        5. **الهدف الوطني:**
        • شرح مختصر لا يتجاوز سطرين فقط.
        • يُولد بناءً على فكرة المشروع الفعلية والمجال المستخدم.
        • يوضّح كيف يدعم المشروع فعليًا التوجه الوطني الذي حدده المستخدم (مثل: دعم التعليم الرقمي، تعزيز نمط الحياة الصحي...).
        • يجب ربطه بـ رؤية المملكة العربية السعودية 2030.
        • يُمنع تثبيت أو نسخ هدف عام لا يعكس الفكرة الفعلية.
        
        6. **شرح الخوارزميات:**
        • شرح علمي مبسط لكل خوارزمية مستخدمة (مثل: Merge Sort، K-Means).
        • توضيح كيف تُستخدم فعليًا داخل النظام.
        • إرفاق مثال تطبيقي لبيان فائدتها.
        
        7. **لغات البرمجة، التقنيات، وخوارزميات الذكاء الاصطناعي:**
        • لكل لغة أو تقنية: شرح لا يتجاوز سطرًا واحدًا يوضح وظيفتها داخل المشروع.
        • في حال تم استخدام الذكاء الاصطناعي:
        o تحديد نوع الذكاء (تحليل، تصنيف، تنبؤ...)
        o ما الذي يتم تحليله أو تصنيفه داخل النظام
        o شرح مبسط لطريقة عمل الخوارزمية
        o مثال تطبيقي يوضح فائدة الخوارزمية
        
        8. **التكامل مع الأنظمة الأخرى:**
        • إذا لم يتم تحديد نوع التكامل من قبل المستخدم، يُتجاهل هذا القسم تمامًا.
        • إذا تم تحديد نوع التكامل (مثل: API خارجي، ذكاء اصطناعي...):
        o شرح وظيفة التكامل عمليًا (مثل: تبادل بيانات، مزامنة حسابات، استخدام نماذج خارجية).
        o تحديد مكان حدوث التكامل داخل النظام.
        o شرح القيمة المضافة من هذا التكامل.
        ________________________________________
        📌 القيود الصارمة (يجب الالتزام بها):
        • ✅ توليد نظام متكامل وظيفيًا، لا مجرد فكرة أو أداة جزئية.
        • ✅ الالتزام التام بجميع الأدوات، اللغات، والتقنيات المذكورة فقط.
        • ✅ لا يُسمح بتعديل أو تجاوز أي قيمة صرّح بها المستخدم.
        • ✅ يُسمح باقتراح القيم فقط في حال غيابها.
        • ✅ يُمنع تضمين الذكاء الاصطناعي نهائيًا ما لم يُطلب صراحة.
        • ✅ لكل مستخدم: 5 وظائف على الأقل.
        • ✅ عند السماح بتوليد مستخدمين إضافيين: يجب توليد مستخدم فعلي مع وظائفه.
        • ✅ لا يُسمح بتكرار عرض المستخدمين أو وظائفهم.
        • ✅ شرح اللغات لا يتجاوز سطرًا واحدًا.
        • ✅ الهدف الوطني لا يتجاوز سطرين، ويُولّد حسب الفكرة ويرتبط برؤية المملكة 2030.
        • ✅ يجب أن يحتوي المشروع على ميزة فارقة تميز النظام بوضوح عن الأنظمة الأخرى.
        • ✅ عدم وضع اي جمل افتتاحية او جمل ختامية في الرد النهائي
        • ✅ الالتزام بترقيم كل الفقرات والعناوين الرئيسية والعناوين الفرعية والاقسام الداخلية يتم ترقيمها بطريقة منسقة
        • ✅ في حالة ان المتغير integration فارغ او لم يتم ارساله يتم حذف الفقرة رقم 8 من الاجابة نهائيا وهي التكامل مع الانظمة الاخرى وعدم ذكرها
        • ✅ يرجى تنسيق النص واضافة الايموجز والعناوين الرئيسية والفرعية بشكل منسق ع طريقة Markdown
        • ✅ يجب التاكد ان العناويين الرئيسية بحجم وخط منسقين يظهرون انهم عناويين رئيسية كذلك العناوين الفرعية والترقيم
        * ✅ هذا مثال توضيحي للشكل النهائي للرد المنتظر منك مع العلم ان فقرة 8 التكامل مع الانظمة الاخرى يتم اضافتها اذا تم ذكر ذلك ويتم حذفها اذا كانت قيمة integration فارغة او غير موجودة
        * ✅ برجاء عدم وضع جمل افتتاحية او ختامية يجب الالتزام بالتركيز ع توليد الفكرة مباشرة
        * ✅ يجب توليد قواعد البيانات المقترحة و التقنيات المستخدمة اذا كانت هذه القيم فارغة او لم يتم عرضها من المستخدم "technologies": "","database_type": ""
        * ✅ يجب استخدام الفكرة الاولية او initial_idea اللتي ارسلها المستخدم في توليد فكرة المشروع )مهمة جدا(
        _______________________________________
        هذه البيانات اللتي يتم بناءا عليها بناء الفكرة بكل تفاصيلها. الرجاء الالتزام بهذه المعطيات فقط
        ## 📂 البيانات المتوفرة:
        تفاصيل الفكرة :

        نوع الدراسة: {data.get("study_type", "")}
        التخصص: {data.get("specialization", "")}
        المجال: {data.get("field", "")}
        التوجه الوطني: {data.get("national_direction", "")}
        خوارزميات المشروع: {data.get("project_algorithms", "")}
        الفئة المستهدفة: {data.get("target_audience", "")}
        نوع النظام: {data.get("system_type", "")}
        التكامل مع أنظمة أخرى: {data.get("integration", "")}
        لغات البرمجة: {data.get("programming_languages", "")}
        التقنيات المستخدمة: {data.get("technologies", "")}
        نوع قواعد البيانات: {data.get("database_type", "")}
        نوع البيانات المستخدمة في الذكاء الاصطناعي: {data.get("ai_data_type", "")}
        أنواع الاستخدام للذكاء الاصطناعي: {data.get("ai_use_types", "")}
        الخوارزميات المقترحة للذكاء الاصطناعي: {data.get("ai_algorithms", "")}
        ملاحظات المستخدم أو وصفه الأولي: {data.get("initial_idea", "")}
        توليد مستخدمين اخرين غير الفئة المستهدفة المذكورة في نطاق فكرة المشروع : {data.get("allow_suggested_users", "")}

        هذا مثال حي يمكنك الاستفادة منه
        1. **العنوان المقترح:**
        نظام "بيانات بلس" لتحسين جودة قواعد البيانات 🌟
        
        2. **وصف تفصيلي للفكرة مع أمثلة:**
        نظام "بيانات بلس" 📊 هو تطبيق جوال مبتكر صُمم خصيصًا لمعالجة المشكلات الشائعة في جودة قواعد البيانات، مثل التكرار، البيانات غير المتسقة، والأخطاء الإملائية. هذه المشكلات تؤثر سلبًا على موثوقية البيانات وصحة القرارات المتخذة بناءً عليها. يعمل النظام عن طريق استقبال البيانات، معالجتها، ثم تحليلها لاكتشاف هذه المشاكل، وتقديم حلول مقترحة لتحسينها.
        يستطيع محللو البيانات وصناع القرار استخدام النظام لتحليل مجموعات بيانات مختلفة، مثل بيانات العملاء 🧑‍🤝‍🧑 أو المنتجات 🛍️، وتصحيح الأخطاء بشكل فعال لضمان دقة المعلومات.
        مثال 1: يمكن لشركة تجزئة 🏪 استخدام "بيانات بلس" لتنظيف قاعدة بيانات عملائها من الإدخالات المكررة أو غير الدقيقة، مما يتيح لهم استهداف العملاء بشكل أفضل وتحسين حملاتهم التسويقية 📈.
        مثال 2: سيناريو آخر هو استخدامه من قبل مؤسسة مالية 💰 للتحقق من سلامة بيانات المعاملات، وتقليل الأخطاء التي قد تؤدي إلى خسائر مالية أو عدم امتثال للوائح.
        
        3. **المستخدمون ووظائفهم في النظام:**
        **مدير النظام ⚙️:**
        إدارة حسابات المستخدمين (إنشاء، تعديل، حذف).
        مراقبة أداء النظام العام.
        تكوين إعدادات النظام ومعايير جودة البيانات.
        الوصول إلى سجلات النظام وتقارير التدقيق.
        إدارة التراخيص والصلاحيات للمستخدمين.
        
        **محلل البيانات 🔬:**
        تحميل وتنزيل مجموعات البيانات.
        بدء عمليات تحليل جودة البيانات.
        مراجعة تقارير جودة البيانات وتوصيات التصحيح.
        تطبيق التعديلات المقترحة على البيانات.
        جدولة عمليات تحليل البيانات الدورية.
        
        **صانع القرار 🎯:**
        عرض لوحات المعلومات المخصصة لجودة البيانات.
        مراجعة التقارير الموجزة عن سلامة البيانات.
        تقييم تأثير جودة البيانات على مؤشرات الأداء الرئيسية.
        طلب تحليلات إضافية لجودة البيانات.
        تلقي إشعارات حول التغييرات الهامة في جودة البيانات.
        
        4. **مميزات المشروع:**
        معالجة تلقائية للبيانات 🤖: يقوم النظام بمعالجة البيانات بشكل تلقائي فور استقبالها، مما يقلل من التدخل اليدوي ويوفر الوقت والجهد.
        تقارير تفصيلية قابلة للتخصيص 📝: يوفر النظام تقارير شاملة عن جودة البيانات، يمكن تخصيصها لتناسب احتياجات المستخدمين المختلفة، مع إمكانية تصديرها بتنسيقات متعددة.
        إدارة صلاحيات دقيقة 🔑: يدعم النظام نظام صلاحيات متعدد المستويات يضمن وصول كل مستخدم إلى البيانات والوظائف المسموح بها فقط، مما يعزز أمان البيانات.
        تتبع حالة العمليات ⏳: يمكن للمستخدمين تتبع حالة عمليات تحليل ومعالجة البيانات في الوقت الفعلي، وتلقي إشعارات عند اكتمالها أو عند حدوث أخطاء.
        ميزة "علامة الجودة الرقمية" 🏅:
        الهدف منها: تهدف هذه الميزة إلى تزويد المستخدمين (خاصة صناع القرار) بمقياس مرئي وموثوق لجودة مجموعات البيانات التي يتم معالجتها داخل النظام. إنها بمثابة شهادة لجودة البيانات بعد إتمام عمليات التحليل والتصحيح.
        كيف تعمل داخل النظام: بعد اكتمال عملية تحليل وتصحيح مجموعة بيانات معينة، يقوم النظام بتقييم شامل لجودة البيانات بناءً على معايير محددة مسبقًا (مثل نسبة البيانات المكررة، نسبة البيانات المفقودة، مدى التناسق). بناءً على هذا التقييم، يتم منح "علامة جودة رقمية" لمجموعة البيانات، والتي يمكن أن تكون على شكل "ممتاز"، "جيد جدًا"، "مقبول"، أو "ضعيف"، مع عرض تفاصيل النقاط التي تم تقييمها.
        أثرها على المستخدم أو سمعة النظام: تزيد هذه الميزة من ثقة المستخدم في البيانات التي يتعامل معها، حيث يمكنهم الاعتماد على هذه العلامة كمؤشر سريع وموثوق لجودة البيانات. كما تعزز من سمعة النظام كمصدر موثوق به لتحسين جودة البيانات، مما يجذب المزيد من المستخدمين.
        سبب تميزها عن بقية الأنظمة المشابهة: تتميز هذه الميزة بكونها تقدم تقييمًا مرئيًا ومصنفًا لجودة البيانات، بدلاً من مجرد عرض تقارير تفصيلية تتطلب تحليلًا يدويًا. إنها توفر للمستخدم لمحة سريعة وشاملة عن حالة البيانات، وتساهم في بناء ثقة أكبر في النظام كأداة لتحسين جودة البيانات بشكل ملموس.
        
        5. **الهدف الوطني:**
        يدعم المشروع التوجه الوطني المتمثل في تعزيز الاستدامة الرقمية ♻️ عبر تحسين وتطوير قواعد البيانات، مما يسهم في تحقيق ركائز رؤية المملكة العربية السعودية 2030 🇸🇦 المتعلقة ببناء اقتصاد رقمي مزدهر ومجتمع معلوماتي معرفي.
        
        6. **شرح الخوارزميات:**
        تحليل الشكل العام (Pattern Recognition) 🔎:
        شرح علمي مبسط: هي عملية اكتشاف الأنماط أو العلاقات المتكررة في البيانات. تهدف إلى التعرف على تشابهات أو اختلافات ضمن مجموعات كبيرة من البيانات.
        الاستخدام داخل النظام: تُستخدم هذه الخوارزمية في "بيانات بلس" لاكتشاف الأنماط غير المتسقة في البيانات، مثل اختلاف تنسيقات التاريخ أو الأرقام، أو الأخطاء الإملائية الشائعة في الأسماء أو العناوين.
        مثال تطبيقي: إذا كانت قاعدة البيانات تحتوي على أسماء مدن مكتوبة بأكثر من شكل (مثلاً: "الرياض" و "رياض" و "Riyadh")، فإن تحليل الشكل العام سيكتشف هذه التباينات ويقترح توحيدها إلى شكل واحد، مما يحسن من جودة البيانات ويجعلها أكثر قابلية للتحليل.
        شجرة البحث الثنائية (Binary Search Tree - BST) 🌳:
        شرح علمي مبسط: هي هيكل بيانات شجري منظم، حيث تكون قيمة كل عقدة أكبر من جميع قيم العقد في فرعها الأيسر وأقل من جميع قيم العقد في فرعها الأيمن. هذا التنظيم يسهل عمليات البحث، الإضافة، والحذف بكفاءة عالية.
        الاستخدام داخل النظام: تُستخدم شجرة البحث الثنائية في "بيانات بلس" لتحسين كفاءة عمليات البحث عن البيانات المكررة أو المتشابهة. عند تحليل مجموعة بيانات كبيرة، يمكن تخزين العناصر في شجرة بحث ثنائية، مما يجعل عملية التحقق من وجود تكرارات أسرع بكثير.
        مثال تطبيقي: عند تنظيف قائمة بريدية طويلة جدًا من عناوين البريد الإلكتروني المكررة، يمكن إضافة كل عنوان بريد إلكتروني إلى شجرة بحث ثنائية. إذا حاول النظام إضافة عنوان موجود بالفعل، فسيتم اكتشافه بسرعة، مما يوفر وقتًا وجهدًا كبيرًا مقارنة بالبحث الخطي في القائمة بأكملها.
        
        7. **لغات البرمجة، التقنيات، وخوارزميات الذكاء الاصطناعي:**
        **لغات البرمجة 💻:**
        Swift: تُستخدم لتطوير تطبيق الجوال على منصة iOS لتقديم واجهة مستخدم سريعة وفعالة.
        Kotlin: تُستخدم لتطوير تطبيق الجوال على منصة Android لضمان التوافق والأداء الأمثل.
        Python: تُستخدم في الواجهة الخلفية لمعالجة البيانات وتطبيق الخوارزميات، وذلك لغناها بالمكتبات المتخصصة في تحليل البيانات.
       ** التقنيات المستخدمة 🛠️:**
        RESTful APIs: تُستخدم لتمكين الاتصال وتبادل البيانات بين تطبيق الجوال والواجهة الخلفية للنظام بشكل فعال وآمن.
        Docker: تُستخدم لحزم ونشر مكونات النظام كحاويات مستقلة، مما يضمن قابلية النقل وسهولة الإدارة.
        Git: يُستخدم لإدارة إصدارات الكود البرمجي والتعاون بين المطورين بشكل فعال.
       ** نوع قواعد البيانات 🗄️:**
        PostgreSQL: تُستخدم كقاعدة بيانات علائقية قوية وموثوقة لتخزين البيانات المنظمة وإدارة معلومات المستخدمين والعمليات.

        8. **التكامل مع الأنظمة الأخرى:**
        • إذا لم يتم تحديد نوع التكامل من قبل المستخدم، يُتجاهل هذا القسم تمامًا.
        • إذا تم تحديد نوع التكامل (مثل: API خارجي، ذكاء اصطناعي...):
        o شرح وظيفة التكامل عمليًا (مثل: تبادل بيانات، مزامنة حسابات، استخدام نماذج خارجية).
        o تحديد مكان حدوث التكامل داخل النظام.
        o شرح القيمة المضافة من هذا التكامل.

        """
    return prompt

# ----------------- Gemini Request -----------------
def ask_gemini(prompt):
    response = gemini_model.generate_content(prompt)
    return response.text.strip()

# ----------------- Extract JSON from Gemini Response -----------------
def extract_json_from_response(text):
    match = re.search(r"\{[\s\S]*\"generated_values\"[\s\S]*?\}", text)
    if match:
        try:
            import json
            raw_json = match.group(0)
            return json.loads(raw_json).get("generated_values", {})
        except Exception:
            return {}
    return {}

# ----------------- Extract National Direction Brief -----------------
def extract_national_direction_brief(text):
    pattern = r"(?:أي أن الهدف هو:|الهدف الوطني(?: هو)?:)([\s\S]{30,300})"
    match = re.search(pattern, text)
    if match:
        return match.group(1).strip()
    return None


# ----------------- API Endpoint -----------------
@app.route("/ping", methods=["GET"])
def ping():
    return jsonify({"message": "Server is awake"}), 200


# ----------------- API Endpoint -----------------
@app.route("/generate", methods=["POST"])
def generate():
    data = request.json or {}
    missing_required = [field for field in MANDATORY_FIELDS if not data.get(field)]

    if missing_required:
        return jsonify({
            "status": "error",
            "message": f"Missing mandatory fields: {', '.join(missing_required)}",
            "required_fields": MANDATORY_FIELDS,
            "optional_fields": OPTIONAL_FIELDS
        }), 400

    missing_optional = [field for field in OPTIONAL_FIELDS if not data.get(field)]
    prompt = build_prompt(data, missing_optional)
    gemini_output = ask_gemini(prompt)
    generated_values = extract_json_from_response(gemini_output)
    national_brief = extract_national_direction_brief(gemini_output)

    return jsonify({
        "status": "success",
        "message": "Project generated successfully",
        "missing_fields": missing_optional,
        "project_markdown": gemini_output,
        "generated_values": generated_values,
        "national_direction_brief": national_brief
    })

# ----------------- Run App -----------------
if __name__ == "__main__":
    print(f"App running at: {public_url}")
    app.run(port=5000)


App running at: https://a23a-34-44-25-167.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


حاضر، إليك نظام متكامل لمشروع تخرجك بناءً على المعطيات التي قدمتها، مع الالتزام الكامل بالقيود والمواصفات المطلوبة:

## 1. **العنوان المقترح:**

"عين الصقر" 🦅: نظام ذكي لتحليل سلوك المرور وتعزيز السلامة

## 2. **وصف تفصيلي للفكرة مع أمثلة:**

"عين الصقر" 🦅 هو نظام ويب مبتكر يعتمد على الذكاء الاصطناعي لتحليل سلوك المشاة والسائقين بهدف تحسين السلامة المرورية وتقليل الحوادث. يستخدم النظام كاميرات المراقبة المرورية المنتشرة في المدن لجمع بيانات الفيديو والصور، ثم يقوم بتحليلها لتحديد المخالفات المرورية المحتملة، وأنماط القيادة الخطرة، وسلوك المشاة غير الآمن.

**طريقة الاستخدام:**

1.  يتم جمع بيانات الفيديو والصور من كاميرات المراقبة المرورية.
2.  تُرسل البيانات إلى النظام، حيث تتم معالجتها باستخدام خوارزميات الذكاء الاصطناعي.
3.  يقوم النظام بتحليل البيانات وتحديد المخالفات المرورية، مثل تجاوز السرعة، عدم الالتزام بالإشارات الضوئية، واستخدام الهاتف أثناء القيادة.
4.  يحلل النظام أيضًا سلوك المشاة، مثل عبور الطريق من أماكن غير مخصصة، أو عدم الانتباه لحركة المرور.
5.  تُعرض النتائج في لوحات

حاضر، إليك نظام متكامل لمشروع تخرجك بناءً على المعطيات التي قدمتها، مع الالتزام الكامل بالقيود والمواصفات المطلوبة:

## 1. **العنوان المقترح:**

"عين الصقر" 🦅: نظام ذكي لتحليل سلوك المرور وتعزيز السلامة

## 2. **وصف تفصيلي للفكرة مع أمثلة:**

"عين الصقر" 🦅 هو نظام ويب مبتكر يعتمد على الذكاء الاصطناعي لتحليل سلوك المشاة والسائقين بهدف تحسين السلامة المرورية وتقليل الحوادث. يستخدم النظام كاميرات المراقبة المرورية المنتشرة في المدن لجمع بيانات الفيديو والصور، ثم يقوم بتحليلها لتحديد المخالفات المرورية المحتملة، وأنماط القيادة الخطرة، وسلوك المشاة غير الآمن.

**طريقة الاستخدام:**

1.  يتم جمع بيانات الفيديو والصور من كاميرات المراقبة المرورية.
2.  تُرسل البيانات إلى النظام، حيث تتم معالجتها باستخدام خوارزميات الذكاء الاصطناعي.
3.  يقوم النظام بتحليل البيانات وتحديد المخالفات المرورية، مثل تجاوز السرعة، عدم الالتزام بالإشارات الضوئية، واستخدام الهاتف أثناء القيادة.
4.  يحلل النظام أيضًا سلوك المشاة، مثل عبور الطريق من أماكن غير مخصصة، أو عدم الانتباه لحركة المرور.
5.  تُعرض النتائج في لوحات معلومات وتقارير تفصيلية للمختصين في المرور.

**الجهات المستفيدة:**

*   إدارات المرور: لتحسين الرقابة المرورية وتحديد المناطق التي تحتاج إلى تدخلات لتحسين السلامة.
*   المشاة والسائقون: من خلال التوعية بالمخاطر المرورية وتعزيز السلوك الآمن.
*   شركات التأمين: لتقييم المخاطر وتحديد أسعار التأمين بناءً على سلوك السائقين.
*   الباحثون: لدراسة أنماط المرور وتطوير حلول مبتكرة لتحسين السلامة.

**سيناريوهات تطبيقية حقيقية:**

*   تحديد المناطق التي تشهد أعلى معدلات حوادث المشاة، واتخاذ إجراءات لتحسين السلامة، مثل إنشاء ممرات عبور آمنة أو تركيب إشارات ضوئية.
*   رصد السائقين الذين يستخدمون الهاتف أثناء القيادة، وإرسال تنبيهات لهم أو اتخاذ إجراءات قانونية بحقهم.
*   تحليل أنماط القيادة الخطرة، مثل القيادة بسرعة عالية أو التجاوز الخاطئ، وتحديد السائقين الذين يحتاجون إلى إعادة تأهيل.

## 3. **المستخدمون ووظائفهم في النظام:**

*   **مدير النظام ⚙️:**
    *   إدارة حسابات المستخدمين وصلاحياتهم.
    *   مراقبة أداء النظام وصحة البيانات.
    *   تكوين إعدادات النظام وتحديث الخوارزميات.
    *   إدارة قواعد البيانات وعمليات النسخ الاحتياطي.
    *   إنشاء تقارير دورية عن أداء النظام.
*   **محلل بيانات المرور 🚦:**
    *   تحميل وتنزيل بيانات الفيديو والصور.
    *   تشغيل خوارزميات التحليل وتصنيف الحوادث.
    *   مراجعة نتائج التحليل وتحديد المخالفات المرورية.
    *   إنشاء تقارير تفصيلية عن الحوادث والمخالفات.
    *   تحديد المناطق التي تحتاج إلى تدخلات لتحسين السلامة.
*   **مشرف كاميرات المراقبة 📹:**
    *   مراقبة حالة الكاميرات والتأكد من عملها بشكل سليم.
    *   تحديد مواقع الكاميرات الجديدة وتوجيهها بشكل فعال.
    *   صيانة الكاميرات وإصلاح الأعطال.
    *   تحديث برامج تشغيل الكاميرات.
    *   تأمين الكاميرات من التخريب أو السرقة.
*   **مسؤول التوعية المرورية 📢:**
    *   إعداد مواد توعوية حول السلامة المرورية.
    *   نشر المواد التوعوية عبر وسائل الإعلام المختلفة.
    *   تنظيم حملات توعية ميدانية.
    *   تحليل نتائج حملات التوعية وقياس فعاليتها.
    *   التواصل مع الجمهور والرد على استفساراتهم.

    **مستخدم فعلي:**
    **اسم المستخدم:** محلل أول للمرور - أحمد العبدلي
        * تحميل البيانات من الكاميرات المرورية
        * تشغيل خوارزميات التحليل للبيانات المرورية
        * إعداد تقارير المخالفات والحوادث المرورية
        * مراجعة دقة التحليلات وإعداد التوصيات
        * تتبع المؤشرات الرئيسية للأداء المروري

## 4. **مميزات المشروع:**

*   **تحليل تلقائي لسلوك المرور 🤖:** يقوم النظام بتحليل بيانات الفيديو والصور بشكل تلقائي، مما يقلل من التدخل اليدوي ويوفر الوقت والجهد.
*   **تحديد دقيق للمخالفات المرورية 🎯:** يستخدم النظام خوارزميات متقدمة لتحديد المخالفات المرورية بدقة عالية، مما يقلل من الأخطاء ويزيد من موثوقية النتائج.
*   **تنبيهات فورية للحوادث المحتملة 🚨:** يمكن للنظام إرسال تنبيهات فورية إلى الجهات المختصة عند اكتشاف حوادث محتملة، مما يتيح لهم التدخل السريع ومنع وقوع الحوادث.
*   **تقارير تفصيلية قابلة للتخصيص 📝:** يوفر النظام تقارير شاملة عن سلوك المرور، يمكن تخصيصها لتلبية احتياجات المستخدمين المختلفة، مع إمكانية تصديرها بتنسيقات متعددة.
*   **ميزة "خريطة المخاطر المرورية" 🗺️:**
    *   **الهدف منها:** تهدف هذه الميزة إلى إنشاء خريطة تفاعلية تعرض المناطق التي تشهد أعلى معدلات حوادث ومخالفات مرورية في المدينة.
    *   **كيف تعمل داخل النظام:** يقوم النظام بتحليل بيانات الحوادث والمخالفات المرورية، وتحديد المناطق التي تشهد أعلى تركيز لهذه الأحداث. ثم يتم عرض هذه المناطق على خريطة تفاعلية باستخدام ألوان مختلفة للإشارة إلى مستوى الخطر (أحمر: خطر مرتفع، أصفر: خطر متوسط، أخضر: خطر منخفض).
    *   **أثرها على المستخدم أو سمعة النظام:** تساعد هذه الميزة إدارات المرور على تحديد المناطق التي تحتاج إلى تدخلات لتحسين السلامة، مثل زيادة الرقابة المرورية، أو تحسين البنية التحتية، أو تنفيذ حملات توعية. كما تساهم في رفع مستوى الوعي لدى الجمهور بالمخاطر المرورية في مناطق معينة. تعزز هذه الميزة من سمعة النظام كأداة فعالة لتحسين السلامة المرورية.
    *   **سبب تميزها عن بقية الأنظمة المشابهة:** تكمن الميزة الفارقة في قدرة النظام على تحديث خريطة المخاطر المرورية بشكل دوري بناءً على البيانات الحديثة، مما يضمن أن الخريطة تعكس الواقع الحالي للمخاطر المرورية في المدينة.

## 5. **الهدف الوطني:**

يدعم المشروع التوجه الوطني نحو التحول الرقمي 🇸🇦 وتعزيز السلامة المرورية، مما يساهم في تحقيق أهداف رؤية المملكة العربية السعودية 2030 🚀 المتعلقة بتحسين جودة الحياة وتوفير بيئة آمنة ومستدامة للجميع.

## 6. **شرح الخوارزميات:**

*   **Decision Trees (أشجار القرار) 🌳:**
    *   **شرح علمي مبسط:** هي خوارزمية تعلم آلي تستخدم لعمليات التصنيف والتنبؤ. تعمل عن طريق تقسيم البيانات إلى فروع بناءً على سلسلة من القرارات، حيث يمثل كل فرع قرارًا وكل ورقة (نهاية الفرع) تمثل النتيجة.
    *   **الاستخدام داخل النظام:** تُستخدم أشجار القرار لتصنيف سلوك السائقين والمشاة بناءً على مجموعة من العوامل، مثل السرعة، المسافة بين المركبات، وموقع المشاة.
    *   **مثال تطبيقي:** يمكن استخدام شجرة القرار لتحديد ما إذا كان السائق يقود بشكل متهور (تصنيف) بناءً على سرعته وتجاوزه للإشارات الضوئية.
*   **Random Forest (الغابة العشوائية) 🌲:**
    *   **شرح علمي مبسط:** هي خوارزمية تعلم آلي تعتمد على تجميع نتائج العديد من أشجار القرار لعمل تنبؤات أكثر دقة. تعمل عن طريق إنشاء مجموعة من أشجار القرار بشكل عشوائي، ثم تجميع نتائجها للحصول على النتيجة النهائية.
    *   **الاستخدام داخل النظام:** تُستخدم الغابة العشوائية لتحسين دقة تصنيف سلوك السائقين والمشاة، وتقليل الأخطاء الناتجة عن استخدام شجرة قرار واحدة.
    *   **مثال تطبيقي:** يمكن استخدام الغابة العشوائية لتحسين دقة تحديد السائقين الذين يستخدمون الهاتف أثناء القيادة، وذلك من خلال تجميع نتائج العديد من أشجار القرار التي تعتمد على عوامل مختلفة، مثل حركة العينين، وموقع اليدين، وسرعة المركبة.

## 7. **لغات البرمجة، التقنيات، وخوارزميات الذكاء الاصطناعي:**

*   **لغات البرمجة 💻:**
    *   Python: تُستخدم في الواجهة الخلفية لتطوير خوارزميات الذكاء الاصطناعي ومعالجة البيانات.
    *   JavaScript: تُستخدم في الواجهة الأمامية لتطوير واجهة المستخدم التفاعلية ولوحات المعلومات.
*   **التقنيات المستخدمة 🛠️:**
    *   Flask: تُستخدم كإطار عمل لتطوير الواجهة الخلفية للنظام وتوفير واجهات برمجة التطبيقات (APIs).
    *   React: تُستخدم كإطار عمل لتطوير الواجهة الأمامية للنظام وتوفير تجربة مستخدم سلسة.
    *   PostgreSQL: تُستخدم كقاعدة بيانات لتخزين بيانات الفيديو والصور، وبيانات المستخدمين، ونتائج التحليل.
*   **أنواع الاستخدام للذكاء الاصطناعي 🧠:**
    *   **تصنيف:** تصنيف سلوك السائقين والمشاة إلى فئات مختلفة (مثل: قيادة متهورة، عبور غير آمن).
    *   **تحليل السلوك:** تحليل أنماط القيادة والمشي لتحديد المخاطر المحتملة.
*   **الخوارزميات المقترحة للذكاء الاصطناعي 🤖:**
    *   **CNN (Convolutional Neural Networks):** تُستخدم لتحليل الصور والفيديوهات وتحديد الأجسام (مثل: المركبات، المشاة، الإشارات الضوئية).
    *   **YOLOv5:** تُستخدم للكشف عن الأجسام في الصور والفيديوهات في الوقت الفعلي، وتحديد مواقعها بدقة عالية.
    *   **تحليل:** تحليل صور وبيانات سلوك السائقين والمشاة لتحديد المخالفات المرورية المحتملة.

## 8. **التكامل مع الأنظمة الأخرى:**

*   **وظيفة التكامل عمليًا:** استخدام نماذج الذكاء الاصطناعي الخارجية (AI)
*   **مكان حدوث التكامل داخل النظام:** عند تحليل بيانات الفيديو والصور، يتم استدعاء نماذج CNN و YOLOv5 المنشأة مسبقًا أو المدربة على مجموعات بيانات كبيرة لتحسين دقة الكشف عن الأجسام وتحديد المخالفات المرورية.
*   **القيمة المضافة من هذا التكامل:** يوفر هذا التكامل القدرة على الاستفادة من أحدث التطورات في مجال الذكاء الاصطناعي وتحسين أداء النظام بشكل مستمر دون الحاجة إلى تطوير نماذج الذكاء الاصطناعي من الصفر. كما يتيح للنظام التعرف على مجموعة واسعة من الأجسام والمخالفات المرورية بدقة عالية، مما يزيد من فعالية النظام في تحسين السلامة المرورية.

INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 11:11:52] "POST /generate HTTP/1.1" 200 -


...

1.  **العنوان المقترح:**
    🚦 "سالم": نظام ويب متكامل لإدارة السلامة المرورية وتحليل الحوادث 🚧

2.  **وصف تفصيلي للفكرة مع أمثلة:**
    نظام "سالم" 🌐 هو منصة ويب شاملة تهدف إلى تحسين السلامة المرورية وتقليل الحوادث من خلال توفير أدوات متكاملة لتحليل البيانات، إدارة المخاطر، والتوعية المرورية. يستهدف النظام المحللين المروريين، ويوفر لهم واجهة سهلة الاستخدام للوصول إلى البيانات المرورية الهامة، وتحليلها، واستخلاص النتائج التي تساعد في اتخاذ قرارات مستنيرة.

    **سيناريوهات تطبيقية:**
    *   **تحليل الحوادث:** يقوم النظام بتجميع بيانات الحوادث المرورية من مصادر مختلفة (مثل الشرطة، المستشفيات، وشركات التأمين) وتحليلها لتحديد الأسباب الرئيسية للحوادث، والمواقع الأكثر خطورة.
    *   **إدارة المخاطر:** يتيح النظام للمحللين المروريين تحديد وتقييم المخاطر المرورية المحتملة، وتطوير خطط للحد منها.
    *   **التوعية المرورية:** يوفر النظام أدوات لإنشاء ونشر مواد توعوية مرورية تستهدف فئات مختلفة من الجمهور، مثل السائقين، المشاة، وراكبي الدراجات.

    **أمثلة:**
    *   يمكن لبلدية مدينة الرياض استخدام "سالم" لتحليل بيانات الحوادث المرورية في المدينة، وتحديد التقاطعات الأكثر خطورة، واتخاذ إجراءات لتحسين السلامة فيها.
    *   يمكن لوزارة النقل استخدام "سالم" لتقييم تأثير مشاريع الطرق الجديدة على السلامة المرورية، واتخاذ قرارات بشأن تصميم الطرق وتنفيذها.
    *   يمكن لإدارة المرور استخدام "سالم" لإنشاء حملات توعية مرورية تستهدف السائقين المتهورين، وتوعيتهم بمخاطر السرعة الزائدة وعدم الالتزام بقواعد المرور.

3.  **المستخدمون ووظائفهم في النظام:**
    *   **مدير النظام ⚙️:**
        1.  إدارة حسابات المستخدمين وصلاحياتهم.
        2.  مراقبة أداء النظام وتحديد المشاكل الفنية وإصلاحها.
        3.  تكوين إعدادات النظام وتحديثه.
        4.  إدارة النسخ الاحتياطية للبيانات واستعادتها عند الحاجة.
        5.  إنشاء تقارير دورية عن استخدام النظام وأدائه.
    *   **محلل مروري 🚦:**
        1.  تحميل وتنزيل البيانات المرورية من مصادر مختلفة.
        2.  تحليل البيانات المرورية باستخدام أدوات التحليل المتاحة في النظام.
        3.  تحديد الأسباب الرئيسية للحوادث المرورية والمواقع الأكثر خطورة.
        4.  تطوير خطط للحد من الحوادث المرورية وتحسين السلامة.
        5.  إنشاء تقارير عن نتائج التحليل وتقديم توصيات لتحسين السلامة المرورية.

4.  **مميزات المشروع:**
    *   **تحليل متكامل للبيانات:** يوفر النظام أدوات متكاملة لتحليل البيانات المرورية من مصادر مختلفة، مما يتيح للمحللين المروريين الحصول على صورة شاملة عن الوضع المروري.
    *   **إدارة المخاطر:** يتيح النظام للمحللين المروريين تحديد وتقييم المخاطر المرورية المحتملة، وتطوير خطط للحد منها.
    *   **التوعية المرورية:** يوفر النظام أدوات لإنشاء ونشر مواد توعوية مرورية تستهدف فئات مختلفة من الجمهور.
    *   **تقارير دورية:** يقوم النظام بإنشاء تقارير دورية عن الوضع المروري، مما يساعد متخذي القرار على اتخاذ قرارات مستنيرة.
    *   **ميزة "خريطة المخاطر المرورية التفاعلية" 🗺️:**
        *   **الهدف منها:** تهدف هذه الميزة إلى توفير تصور مرئي وتفاعلي للمخاطر المرورية في منطقة معينة، مما يساعد المحللين المروريين وصناع القرار على تحديد المواقع الأكثر خطورة واتخاذ الإجراءات اللازمة لتحسين السلامة فيها.
        *   **كيف تعمل داخل النظام:** يقوم النظام بإنشاء خريطة تفاعلية للمنطقة المستهدفة، ويقوم بتلوين المواقع المختلفة على الخريطة بناءً على مستوى المخاطر المرورية فيها (مثل عدد الحوادث، شدة الحوادث، عدد الإصابات، عدد الوفيات). يمكن للمستخدم النقر على أي موقع على الخريطة لعرض تفاصيل المخاطر المرورية فيه، مثل الأسباب الرئيسية للحوادث، وأنواع الحوادث الأكثر شيوعًا، والفئات الأكثر عرضة للإصابة.
        *   **أثرها على المستخدم أو سمعة النظام:** تزيد هذه الميزة من فعالية النظام في تحديد المخاطر المرورية واتخاذ الإجراءات اللازمة للحد منها. كما تعزز من سمعة النظام كمصدر موثوق به للمعلومات المرورية، مما يجذب المزيد من المستخدمين.
        *   **سبب تميزها عن بقية الأنظمة المشابهة:** تتميز هذه الميزة بكونها توفر تصورًا مرئيًا وتفاعليًا للمخاطر المرورية، بدلاً من مجرد عرض تقارير تفصيلية تتطلب تحليلًا يدويًا. إنها توفر للمستخدم لمحة سريعة وشاملة عن الوضع المروري، وتساهم في اتخاذ قرارات مستنيرة لتحسين السلامة المرورية.

5.  **الهدف الوطني:**
    يدعم المشروع التوجه الوطني نحو التحول الرقمي 🚀 في قطاع النقل والمرور، مما يساهم في تحقيق أهداف رؤية المملكة 2030 🇸🇦 المتعلقة بتحسين جودة الحياة وتوفير بيئة آمنة ومستدامة للمواطنين والمقيمين.

6.  **شرح الخوارزميات:**
    *   **خوارزمية التجميع (Clustering):**
        *   **شرح علمي مبسط:** هي عملية تقسيم مجموعة من البيانات إلى مجموعات (clusters) بحيث تكون البيانات داخل كل مجموعة متشابهة قدر الإمكان، والبيانات بين المجموعات مختلفة قدر الإمكان.
        *   **الاستخدام داخل النظام:** تُستخدم خوارزمية التجميع في "سالم" لتحديد المواقع التي تتشابه في خصائصها المرورية، مثل عدد الحوادث، شدة الحوادث، وأنواع الحوادث.
        *   **مثال تطبيقي:** يمكن استخدام خوارزمية التجميع لتحديد مجموعات من التقاطعات التي تتشابه في عدد الحوادث وأنواع الحوادث، ثم تحليل هذه المجموعات لتحديد الأسباب الرئيسية للحوادث واتخاذ الإجراءات اللازمة لتحسين السلامة فيها.
    *   **خوارزمية التصنيف (Classification):**
        *   **شرح علمي مبسط:** هي عملية تحديد الفئة التي تنتمي إليها بيانات معينة بناءً على مجموعة من الخصائص.
        *   **الاستخدام داخل النظام:** تُستخدم خوارزمية التصنيف في "سالم" لتصنيف الحوادث المرورية بناءً على الأسباب الرئيسية لها، مثل السرعة الزائدة، عدم الالتزام بقواعد المرور، أو القيادة تحت تأثير الكحول.
        *   **مثال تطبيقي:** يمكن استخدام خوارزمية التصنيف لتصنيف الحوادث المرورية بناءً على الأسباب الرئيسية لها، ثم تحليل هذه الفئات لتحديد الأسباب الأكثر شيوعًا للحوادث واتخاذ الإجراءات اللازمة للحد منها.

7.  **لغات البرمجة، التقنيات، وخوارزميات الذكاء الاصطناعي:**
    *   **لغات البرمجة 💻:**
        *   Python: تُستخدم في الواجهة الخلفية لتطوير الخوارزميات ومعالجة البيانات وتحليلها.
        *   JavaScript: تُستخدم في الواجهة الأمامية لتطوير واجهة المستخدم التفاعلية.
    *   **التقنيات المستخدمة 🛠️:**
        *   Flask: تُستخدم كإطار عمل (framework) لتطوير الواجهة الخلفية للنظام.
        *   React: تُستخدم كمكتبة (library) لتطوير واجهة المستخدم الأمامية للنظام.
        *   PostgreSQL: تُستخدم كقاعدة بيانات لتخزين البيانات المرورية وبيانات المستخدمين.

INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 11:19:24] "POST /generate HTTP/1.1" 200 -


1.  **العنوان المقترح:**
    🚦 "سلامتك أولاً": نظام ويب للتوعية المرورية

2.  **وصف تفصيلي للفكرة مع أمثلة:**
    نظام "سلامتك أولاً" هو منصة ويب تفاعلية تهدف إلى رفع مستوى الوعي المروري لدى مختلف فئات المجتمع. يركز النظام على تقديم محتوى تعليمي وتوعوي بطرق مبتكرة وجذابة، معتمداً على أحدث التقنيات في مجال تصميم المواقع وتجربة المستخدم.
    النظام موجه بشكل خاص للمحللين المروريين، حيث يوفر لهم أدوات تحليلية وتقارير مفصلة حول سلوكيات القيادة الشائعة وأسباب الحوادث، مما يساعدهم في اتخاذ قرارات مستنيرة لتحسين السلامة المرورية.
    **أمثلة تطبيقية:**
    *   **مثال 1:** يمكن للمحلل المروري استخدام النظام لتحليل بيانات الحوادث في منطقة معينة، وتحديد الأسباب الرئيسية وراء هذه الحوادث (مثل السرعة الزائدة، عدم الالتزام بإشارات المرور، استخدام الهاتف أثناء القيادة). بناءً على هذا التحليل، يمكنه اقتراح حلول لتحسين السلامة المرورية في تلك المنطقة، مثل زيادة عدد اللوحات الإرشادية، تعديل توقيت إشارات المرور، أو تنفيذ حملات توعية مكثفة.
    *   **مثال 2:** يمكن للمستخدم العادي (سائق، راكب، أو عابر سب

1.  **العنوان المقترح:**
    🚦 "سلامتك أولاً": نظام ويب للتوعية المرورية

2.  **وصف تفصيلي للفكرة مع أمثلة:**
    نظام "سلامتك أولاً" هو منصة ويب تفاعلية تهدف إلى رفع مستوى الوعي المروري لدى مختلف فئات المجتمع. يركز النظام على تقديم محتوى تعليمي وتوعوي بطرق مبتكرة وجذابة، معتمداً على أحدث التقنيات في مجال تصميم المواقع وتجربة المستخدم.
    النظام موجه بشكل خاص للمحللين المروريين، حيث يوفر لهم أدوات تحليلية وتقارير مفصلة حول سلوكيات القيادة الشائعة وأسباب الحوادث، مما يساعدهم في اتخاذ قرارات مستنيرة لتحسين السلامة المرورية.
    **أمثلة تطبيقية:**
    *   **مثال 1:** يمكن للمحلل المروري استخدام النظام لتحليل بيانات الحوادث في منطقة معينة، وتحديد الأسباب الرئيسية وراء هذه الحوادث (مثل السرعة الزائدة، عدم الالتزام بإشارات المرور، استخدام الهاتف أثناء القيادة). بناءً على هذا التحليل، يمكنه اقتراح حلول لتحسين السلامة المرورية في تلك المنطقة، مثل زيادة عدد اللوحات الإرشادية، تعديل توقيت إشارات المرور، أو تنفيذ حملات توعية مكثفة.
    *   **مثال 2:** يمكن للمستخدم العادي (سائق، راكب، أو عابر سبيل) استخدام النظام للتعرف على قواعد المرور الأساسية، وأفضل الممارسات للقيادة الآمنة، وكيفية التعامل مع الحالات الطارئة على الطريق. يمكنه أيضاً المشاركة في استطلاعات الرأي والاختبارات التفاعلية لتقييم مستوى وعيه المروري، وتلقي توصيات لتحسينه.

3.  **المستخدمون ووظائفهم في النظام:**

    *   **مدير النظام ⚙️:**
        1.  إدارة حسابات المستخدمين (إنشاء، تعديل، حذف).
        2.  تحديد صلاحيات الوصول للمستخدمين.
        3.  مراقبة أداء النظام وضمان استقراره.
        4.  تحديث محتوى النظام وإضافة مواد توعوية جديدة.
        5.  إنشاء تقارير دورية عن استخدام النظام ومدى فعاليته.

    *   **محلل مروري 🚦:**
        1.  تحليل بيانات الحوادث المرورية واستخلاص النتائج.
        2.  تحديد المناطق الأكثر خطورة على الطرق.
        3.  اقتراح حلول لتحسين السلامة المرورية.
        4.  متابعة تنفيذ الخطط المرورية وتقييم فعاليتها.
        5.  إعداد تقارير دورية عن حالة المرور والسلامة على الطرق.
    *   **مستخدم فعلي: مواطن عادي 👨:**
        1.  مشاهدة مقاطع الفيديو التوعوية.
        2.  قراءة المقالات والنصائح المرورية.
        3.  المشاركة في الاختبارات التفاعلية لتقييم مستوى الوعي.
        4.  الإبلاغ عن المخالفات المرورية أو المشاكل على الطرق.
        5.  تقديم اقتراحات لتحسين السلامة المرورية.

4.  **مميزات المشروع:**

    *   **محتوى تفاعلي وجذاب 🎁:** يقدم النظام محتوى توعويًا متنوعًا (مقاطع فيديو، رسوم بيانية، اختبارات تفاعلية) لجذب انتباه المستخدمين وزيادة تفاعلهم.
    *   **تحليل متعمق لبيانات الحوادث 📊:** يوفر النظام أدوات تحليلية متقدمة للمحللين المروريين، تساعدهم في فهم أسباب الحوادث واقتراح حلول فعالة.
    *   **تقارير دورية مفصلة 📝:** يقوم النظام بإنشاء تقارير دورية عن حالة المرور والسلامة على الطرق، تساعد في اتخاذ القرارات المستنيرة.
    *   **تحديثات مستمرة للمحتوى ✅:** يتم تحديث محتوى النظام بشكل دوري لضمان مواكبته لأحدث التطورات في مجال السلامة المرورية.
    *   **ميزة "تقييم المخاطر الشخصية" 🎯:**
        *   **الهدف منها:** تمكين المستخدمين من تقييم مستوى المخاطر المرورية التي يتعرضون لها بشكل يومي، بناءً على سلوكياتهم وعاداتهم في القيادة أو استخدام الطريق.
        *   **كيف تعمل داخل النظام:** يقوم المستخدم بالإجابة على مجموعة من الأسئلة المتعلقة بسلوكياته المرورية (مثل عدد ساعات القيادة اليومية، مدى الالتزام بقواعد المرور، استخدام الهاتف أثناء القيادة، حالة السيارة)، ثم يقوم النظام بتحليل هذه الإجابات وتقديم تقرير شخصي يوضح مستوى المخاطر التي يتعرض لها، بالإضافة إلى توصيات لتحسين سلوكياته وتقليل هذه المخاطر.
        *   **أثرها على المستخدم أو سمعة النظام:** تزيد هذه الميزة من وعي المستخدم بالمخاطر التي قد لا يكون مدركاً لها، وتشجعه على تغيير سلوكياته لتجنب الحوادث. كما تعزز من سمعة النظام كمنصة موثوقة وفعالة للتوعية المرورية.
        *   **سبب تميزها عن بقية الأنظمة المشابهة:** تتميز هذه الميزة بكونها شخصية وموجهة لكل مستخدم على حدة، حيث تعتمد على تحليل سلوكياته الخاصة لتقديم توصيات مخصصة لتحسين السلامة المرورية. بدلاً من تقديم معلومات عامة، تساعد هذه الميزة المستخدم على فهم المخاطر التي يواجهها بشكل مباشر، وتمنحه الأدوات اللازمة للتعامل معها.

5.  **الهدف الوطني:**
    يدعم المشروع التوجه الوطني نحو التحول الرقمي 🇸🇦 في مجال السلامة المرورية، مما يسهم في تحقيق أهداف رؤية المملكة العربية السعودية 2030 🚀 المتعلقة بتحسين جودة الحياة وتوفير بيئة آمنة ومستدامة للجميع.

6.  **شرح الخوارزميات:**

    *   **خوارزمية البحث الثنائي (Binary Search) 🔎:**
        *   **شرح علمي مبسط:** هي خوارزمية بحث فعالة تستخدم للعثور على عنصر معين في قائمة مرتبة. تعمل عن طريق تقسيم القائمة إلى نصفين بشكل متكرر، ومقارنة العنصر المراد البحث عنه بالعنصر الموجود في منتصف القائمة. إذا كان العنصر المراد البحث عنه أصغر من العنصر الموجود في المنتصف، يتم البحث في النصف الأيسر من القائمة. وإذا كان أكبر، يتم البحث في النصف الأيمن. تستمر هذه العملية حتى يتم العثور على العنصر أو حتى يتم استنفاد جميع عناصر القائمة.
        *   **الاستخدام داخل النظام:** تُستخدم هذه الخوارزمية في نظام "سلامتك أولاً" للبحث عن معلومات محددة في قاعدة البيانات بسرعة وكفاءة. على سبيل المثال، يمكن استخدامها للبحث عن معلومات حول مخالفة مرورية معينة، أو للبحث عن إحصائيات حول الحوادث المرورية في منطقة معينة.
        *   **مثال تطبيقي:** إذا أراد المستخدم البحث عن معلومات حول مخالفة مرورية برقم معين، يمكن استخدام خوارزمية البحث الثنائي للبحث عن هذه المخالفة في قاعدة البيانات المرتبة حسب رقم المخالفة.

    *   **خوارزمية الترتيب السريع (Quick Sort) 🔢:**
        *   **شرح علمي مبسط:** هي خوارزمية ترتيب فعالة تعتمد على مبدأ "فرق تسد". تعمل عن طريق اختيار عنصر محوري من القائمة، ثم تقسيم القائمة إلى قسمين: قسم يحتوي على العناصر الأصغر من العنصر المحوري، وقسم يحتوي على العناصر الأكبر من العنصر المحوري. ثم يتم ترتيب كل قسم من هذين القسمين بشكل متكرر باستخدام نفس الخوارزمية.
        *   **الاستخدام داخل النظام:** تُستخدم هذه الخوارزمية في نظام "سلامتك أولاً" لترتيب البيانات بسرعة وكفاءة. على سبيل المثال، يمكن استخدامها لترتيب قائمة الحوادث المرورية حسب تاريخ وقوعها، أو لترتيب قائمة المخالفات المرورية حسب قيمة الغرامة.
        *   **مثال تطبيقي:** إذا أراد النظام عرض قائمة الحوادث المرورية مرتبة حسب تاريخ وقوعها، يمكن استخدام خوارزمية الترتيب السريع لترتيب هذه القائمة قبل عرضها للمستخدم.

7.  **لغات البرمجة، التقنيات، ونوع قواعد البيانات:**

    *   **لغات البرمجة 💻:**
        *   Python: تُستخدم في الواجهة الخلفية (Back-end) لتطوير الخوارزميات ومعالجة البيانات.
        *   JavaScript: تُستخدم في الواجهة الأمامية (Front-end) لتطوير واجهة المستخدم التفاعلية.

    *   **التقنيات المستخدمة 🛠️:**
        *   Flask: إطار عمل (Framework) خفيف الوزن لتطوير تطبيقات الويب باستخدام Python.
        *   React: مكتبة JavaScript لتطوير واجهات المستخدم الديناميكية والتفاعلية.

    *   **نوع قواعد البيانات 🗄️:**
        *   PostgreSQL: نظام إدارة قواعد بيانات علائقية (RDBMS) قوي وموثوق لتخزين وإدارة بيانات النظام.

INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 11:21:03] "POST /generate HTTP/1.1" 200 -
